<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-Framework" data-toc-modified-id="Python-Framework-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python Framework</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Libraries-and-dependencies" data-toc-modified-id="Libraries-and-dependencies-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Libraries and dependencies</a></span></li><li><span><a href="#Import-packages-and-create-modules" data-toc-modified-id="Import-packages-and-create-modules-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Import packages and create modules</a></span></li></ul></li><li><span><a href="#Data-Import" data-toc-modified-id="Data-Import-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data Import</a></span><ul class="toc-item"><li><span><a href="#Load-Calibration-Data" data-toc-modified-id="Load-Calibration-Data-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Load Calibration Data</a></span></li><li><span><a href="#Import-Local-File" data-toc-modified-id="Import-Local-File-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Import Local File</a></span></li></ul></li><li><span><a href="#Formulas" data-toc-modified-id="Formulas-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Formulas</a></span><ul class="toc-item"><li><span><a href="#Useful-formulas" data-toc-modified-id="Useful-formulas-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Useful formulas</a></span></li><li><span><a href="#Calculator" data-toc-modified-id="Calculator-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Calculator</a></span></li></ul></li><li><span><a href="#AlphaSense-Baseline-Calibration" data-toc-modified-id="AlphaSense-Baseline-Calibration-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>AlphaSense Baseline Calibration</a></span><ul class="toc-item"><li><span><a href="#Parameters" data-toc-modified-id="Parameters-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Correction" data-toc-modified-id="Correction-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Correction</a></span></li></ul></li><li><span><a href="#Data-Export" data-toc-modified-id="Data-Export-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Data Export</a></span><ul class="toc-item"><li><span><a href="#Local-Data-Export" data-toc-modified-id="Local-Data-Export-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Local Data Export</a></span></li></ul></li><li><span><a href="#Exploratory-Data-Analysis" data-toc-modified-id="Exploratory-Data-Analysis-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Exploratory Data Analysis</a></span><ul class="toc-item"><li><span><a href="#Time-Series-Plots" data-toc-modified-id="Time-Series-Plots-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Time Series Plots</a></span></li><li><span><a href="#TODO:-Basic-Sensor-Correlations" data-toc-modified-id="TODO:-Basic-Sensor-Correlations-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>TODO: Basic Sensor Correlations</a></span><ul class="toc-item"><li><span><a href="#Full-Seaborn-Correlogram" data-toc-modified-id="Full-Seaborn-Correlogram-1.5.2.1"><span class="toc-item-num">1.5.2.1&nbsp;&nbsp;</span>Full Seaborn Correlogram</a></span></li><li><span><a href="#Basic-Seaborn-XYPlot" data-toc-modified-id="Basic-Seaborn-XYPlot-1.5.2.2"><span class="toc-item-num">1.5.2.2&nbsp;&nbsp;</span>Basic Seaborn XYPlot</a></span></li></ul></li><li><span><a href="#TODO:-Anomaly-Detection" data-toc-modified-id="TODO:-Anomaly-Detection-1.5.3"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>TODO: Anomaly Detection</a></span></li></ul></li><li><span><a href="#Data-Model" data-toc-modified-id="Data-Model-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Data Model</a></span><ul class="toc-item"><li><span><a href="#Basic-Functionality" data-toc-modified-id="Basic-Functionality-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Basic Functionality</a></span></li><li><span><a href="#Feature-selection-and-data-training-split" data-toc-modified-id="Feature-selection-and-data-training-split-1.6.2"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>Feature selection and data training split</a></span><ul class="toc-item"><li><span><a href="#TODO-Select-variables-here" data-toc-modified-id="TODO-Select-variables-here-1.6.2.1"><span class="toc-item-num">1.6.2.1&nbsp;&nbsp;</span>TODO Select variables here</a></span></li><li><span><a href="#TODO-Preliminary-Checks" data-toc-modified-id="TODO-Preliminary-Checks-1.6.2.2"><span class="toc-item-num">1.6.2.2&nbsp;&nbsp;</span>TODO Preliminary Checks</a></span></li></ul></li><li><span><a href="#TODO-Naive-Linear-Regression" data-toc-modified-id="TODO-Naive-Linear-Regression-1.6.3"><span class="toc-item-num">1.6.3&nbsp;&nbsp;</span>TODO Naive Linear Regression</a></span><ul class="toc-item"><li><span><a href="#Ordinary-Linear-Regression" data-toc-modified-id="Ordinary-Linear-Regression-1.6.3.1"><span class="toc-item-num">1.6.3.1&nbsp;&nbsp;</span>Ordinary Linear Regression</a></span></li><li><span><a href="#Ordinary-Linear-Regression-with-differentiation" data-toc-modified-id="Ordinary-Linear-Regression-with-differentiation-1.6.3.2"><span class="toc-item-num">1.6.3.2&nbsp;&nbsp;</span>Ordinary Linear Regression with differentiation</a></span></li></ul></li><li><span><a href="#TODO-ARIMA(X)-model" data-toc-modified-id="TODO-ARIMA(X)-model-1.6.4"><span class="toc-item-num">1.6.4&nbsp;&nbsp;</span>TODO ARIMA(X) model</a></span></li></ul></li></ul></li><li><span><a href="#R-Framework" data-toc-modified-id="R-Framework-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>R Framework</a></span><ul class="toc-item"><li><span><a href="#Initialise-environment" data-toc-modified-id="Initialise-environment-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initialise environment</a></span></li><li><span><a href="#Install-dependencies" data-toc-modified-id="Install-dependencies-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Install dependencies</a></span></li><li><span><a href="#Load-in-R-libraries" data-toc-modified-id="Load-in-R-libraries-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Load in R libraries</a></span></li><li><span><a href="#Export-Data-to-R-Dataframe" data-toc-modified-id="Export-Data-to-R-Dataframe-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Export Data to R Dataframe</a></span><ul class="toc-item"><li><span><a href="#Renaming-and-timestamp-reading" data-toc-modified-id="Renaming-and-timestamp-reading-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Renaming and timestamp reading</a></span></li></ul></li><li><span><a href="#Data-Exploration" data-toc-modified-id="Data-Exploration-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Data Exploration</a></span><ul class="toc-item"><li><span><a href="#Pairs-Plot" data-toc-modified-id="Pairs-Plot-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>Pairs Plot</a></span></li><li><span><a href="#Coplot" data-toc-modified-id="Coplot-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span>Coplot</a></span></li></ul></li><li><span><a href="#Model-Iterations" data-toc-modified-id="Model-Iterations-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Model Iterations</a></span><ul class="toc-item"><li><span><a href="#Model-Diagnostics" data-toc-modified-id="Model-Diagnostics-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>Model Diagnostics</a></span></li><li><span><a href="#Model-Fit-Plot" data-toc-modified-id="Model-Fit-Plot-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>Model Fit Plot</a></span><ul class="toc-item"><li><span><a href="#Using-Traditional-R" data-toc-modified-id="Using-Traditional-R-2.6.2.1"><span class="toc-item-num">2.6.2.1&nbsp;&nbsp;</span>Using Traditional R</a></span></li><li><span><a href="#Using-interactive-Plot" data-toc-modified-id="Using-interactive-Plot-2.6.2.2"><span class="toc-item-num">2.6.2.2&nbsp;&nbsp;</span>Using interactive Plot</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Python Framework

In [ ]:
from IPython.display import HTML

HTML(
    '''
    <script>
    code_show=true; 
    function code_toggle() {
        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

---
### Libraries and dependencies
Run this cell to install all necessary dependencies

In [ ]:
! pip install pytz==2017.2 fileupload==0.1.2 ipywidgets==6.0.0 pandas==0.20.1 numpy==1.12.1 matplotlib==2.0.2 seaborn==0.8.0
! jupyter nbextension install --py fileupload 
! jupyter nbextension enable --py fileupload
! jupyter nbextension install --py widgetsnbextension 
! jupyter nbextension enable --py widgetsnbextension
! jupyter nbextension install https://rawgit.com/jfbercher/small_nbextensions/master/toc2.zip  --user
#! jupyter nbextension install --py toc2/main 
! jupyter nbextension enable toc2/main
! pip install cufflinks --upgrade
! pip install plotly --upgrade

---
### Import packages and create modules
Run this cell to load in all necessary Python modules

In [ ]:
from IPython.display import display, Markdown, FileLink, FileLinks, clear_output, HTML
from IPython.core.display import HTML
from IPython.display import display, clear_output
from plotly.widgets import GraphWidget
    
import io, pytz, os, time, datetime, fileupload
import ipywidgets as widgets
import pandas as pd
import numpy as np
import cufflinks as cf
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Layout
import plotly.figure_factory as ff
from plotly.figure_factory import create_2d_density
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.tools as tls

sns.set(color_codes=True)
%matplotlib inline
matplotlib.style.use('seaborn-whitegrid')
init_notebook_mode(connected=True)
cf.go_offline()

---
## Data Import

### Load Calibration Data

In [25]:
import pandas as pd

alpha_calData = pd.read_json('https://raw.githubusercontent.com/fablabbcn/smartcitizen-iscape-data/internal_dev/calData/AlphaSense.json', orient='columns', lines = True)
alpha_calData.index = alpha_calData['Serial No']


print display(alpha_calData.describe())
#print alpha_calData.loc[162581712,:]

## Baseline

,Aux Zero Current,Sensitivity 1,Serial No,Zero Current
count,69.000000,69.000000,6.900000e+01,69.000000
mean,-1.658986,-74.043333,1.896450e+08,-10.244783
std,26.354277,471.048491,1.934604e+07,45.344773
min,-71.900000,-489.690000,1.620313e+08,-99.000000
25%,-21.100000,-433.120000,1.625817e+08,-67.200000
50%,8.500000,-352.000000,2.021604e+08,15.400000
75%,19.230000,568.300000,2.041601e+08,23.650000
max,30.270000,621.600000,2.045603e+08,34.050000


None


In [26]:
print alpha_calData

           Aux Zero Current  Sensitivity 1 Sensitivity 2  Serial No Target 1  \
Serial No                                                                      
162031254            -20.80         568.30            na  162031254       CO   
162031257            -18.60         493.10            na  162031257       CO   
162031256            -13.90         601.90            na  162031256       CO   
162581706            -35.30         581.40            na  162581706       CO   
162581707            -46.30         605.00            na  162581707       CO   
162581708            -51.10         597.40            na  162581708       CO   
162581709            -43.20         577.90            na  162581709       CO   
162581710            -33.40         570.00            na  162581710       CO   
162581711            -36.30         564.30            na  162581711       CO   
162581712            -71.90         598.40            na  162581712       CO   
162581713            -46.70         605.

### Import Local File

In [ ]:
start_date = 0
end_date = 0
openData = list()

def _upload():
    
    _upload_widget = fileupload.FileUploadWidget()
    _tz_widget = widgets.Dropdown(options=pytz.common_timezones, value='UTC', description='Timezone: ')
    
    def _cb(change):
        # get file
        decoded = io.StringIO(change['owner'].data.decode('utf-8'))
        filename = change['owner'].filename 
        fileData = io.StringIO(change['new'].decode('utf-8'))
        df = pd.read_csv(fileData, verbose=True, skiprows=[1]).set_index('Time')
          
        # prepare dataframe
        print df.index
        df.index = pd.to_datetime(df.index).tz_localize('UTC').tz_convert(_tz_widget.value)
        df.sort_index(inplace=True)
        df = df.groupby(pd.TimeGrouper(freq='10Min')).aggregate(np.mean)
        df.drop([i for i in df.columns if 'Unnamed' in i], axis=1, inplace=True)
        
        readings[filename] = df[df.index > '2001-01-01T00:00:01Z']
        if start_date > 0: readings[filename] = df[df.index > start_date]
        if end_date > 0: readings[filename] = df[df.index < end_date]
        listFiles(filename)
    
    # widgets
    _upload_widget.observe(_cb, names='data')
    _hb = widgets.HBox([_upload_widget, _tz_widget, widgets.HTML(' ')])
    display(_hb)

def delFile(b):
    clear_output()
    for d in list(b.hbl.children): d.close()
    readings.pop(b.f)

def describeFile(b):
    clear_output()
    display(readings[b.f].describe())
    
def exportFile(b):
    export_dir = 'exports'
    if not os.path.exists(export_dir): os.mkdir(export_dir)
    savePath = os.path.join(export_dir, b.f+'_clean_'+datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%dT%H%M%S')
+'.csv')
    if not os.path.exists(savePath):
        readings[b.f].to_csv(savePath, sep=",")
        display(FileLink(savePath))
    else:
        display(widgets.HTML(' File Already exists!'))
    
def listFiles(filename):
#     clear_output()
    temp = list(fileList.children)
    cb = widgets.Button(icon='close',layout=widgets.Layout(width='30px'))
    cb.on_click(delFile)
    cb.f = filename
    eb = widgets.Button(description='Export processed CSV', layout=widgets.Layout(width='180px'))
    eb.on_click(exportFile)
    eb.f = filename
    sb = widgets.Button(description='describe', layout=widgets.Layout(width='80px'))
    sb.on_click(describeFile)
    sb.f = filename  
    hbl = widgets.HBox([cb, widgets.HTML(' <b>'+filename+'</b> \t'), sb, eb])
    cb.hbl = hbl
    temp.append(hbl)
    fileList.children = temp

readings = {}
display(widgets.HTML('<hr><h3>Select CSV files (remember to change the timezone!)</h3>'))
_upload()
fileList = widgets.VBox([widgets.HTML('<hr>')])
display(fileList)


## Formulas

### Useful formulas

There are formulas for calculating:
- *MICS* = Poly(R, H, T) - **MICS_FORMULA**
- *Alphasense* = f(Curr, Sens, Zero) - **AD_FORMULA**
- *Smoothing* = f(Signal, Window) - **SMOOTH**

In [ ]:
def SMOOTH(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def AD_FORMULA(WE, AE, SensorType, I0WE, I0AE, SENSITIVITY1, SENSITIVITY2, AUX, UNIT1, UNIT2, X1W, X2W, X1A, X2A):
    
    
    MW,BW = LINE_COEFF(X1W, X2W, -1400, 1399)
    MA,BA = LINE_COEFF(X1A, X2A, -1400, 1399)
    
    FACTORWE = LINE(WE,MW,BW) / WE
    FACTORAE = LINE(AE,MA,BA) / AE
    
    # import matplotlib.pyplot as plt
    # plt.plot(FACTORWE)
    # plt.plot(FACTORAE)

    if SensorType == 1 or 2:
        # CO OR NO2
        # CO: BOARD 5 AD_FORMULA(A, B, 1, -69.4, -18.6, 493.1, 0, C, "ppm", "", -220.7, 220.42, -220.45, 220.09)
        # NO2: BOARD U AD_FORMULA(A, B, 1, 31.5, 17.7, -383.7, 0, C, "ppb", "", -220.45, 220.19, -219.98, 219.61)
        # NO2: BOARD 5 AD_FORMULA(A, B, 1, 24.0, 14.2, -385.9, 0, C, "ppb", "", -220.69, 220.58, -220.39, 220.17)
        result = (FACTORWE*WE - I0WE/I0AE*(FACTORAE*AE))/abs(SENSITIVITY1)
        
        if UNIT1 == "ppb":
            result = result*1000
    
    if SensorType== 3:
        # O3
        # BOARD U AD_FORMULA(A, B, 3, 1, 23.65, 18.92, -421.58, -471.6497, C, "ppb", "ppb", -220.69, 220.58, -220.39, 220.17)
        # BOARD 5 AD_FORMULA(A, B, 3, 1, 23.33, 19.86, -433.12, -506.96, C, "ppb", "ppb", -220.69, 220.58, -220.39, 220.17)
    
        if UNIT2 =="ppb":
            result = (FACTORWE*WE - I0WE/I0AE*(FACTORAE*AE) - AUX*SENSITIVITY2/1000)/SENSITIVITY1
        if UNIT2 =="ppm":
            result = (FACTORWE*WE - I0WE/I0AE*(FACTORAE*AE) - AUX*SENSITIVITY2)/SENSITIVITY1
        if UNIT1 == "ppb":
            result = result*1000

    return result

def LINE_COEFF(X1,X2,Y1,Y2):
    a = float(Y2-Y1)/(X2-X1)
    b = Y1-a*X1
    return a,b

def LINE(x,a,b):
    y = [i*a+b for i in x]
    return y

def MICS_FORMULA(Sensor1Type, Sensor1, Sensor2Type, Sensor2, Sensor3Type, Sensor3, Intercept, B, C, D, E, F, G):
    SensorType = [Sensor1Type, Sensor2Type, Sensor3Type]
    Sensor = [Sensor1, Sensor2, Sensor3]
    Sens = Sensor
    for i in range(3):
        if SensorType[i] == "Inverse":
            Sens[i] = 1/Sensor[i]
    
    result =  Intercept + B*Sens[0] +C*Sens[0]*Sens[0] + D*Sens[1] + E*Sens[1]*Sens[1] + F*Sens[2] + G*Sens[2]*Sens[2]

    # MICS_Formula("Inverse", A, "Direct", B, "Direct", C, -1.615e-01 , 210.08064516, -10236.73257, 0, 0, 0, 0)
    return result

def greater(y, val):
    result = np.zeros(len(y))
    for i in range(len(y)):
        if (y[i]>val): result[i]=True
        else: result [i]=False        
    return result

def greaterequal(y, val):
    result = np.zeros(len(y))
    for i in range(len(y)):
        if (y[i]<val): result[i] = False
        else: result [i] = True
    return result

def lower (y, val):
    result = np.zeros(len(y))
    for i in range(len(y)):
        if (y[i]<val): result[i]=True
        else: result [i]=False        
    return result

def lowerequal(y, val):
    result = np.zeros(len(y))
    for i in range(len(y)):
        if (y[i]>val): result[i] = False
        else: result [i] = True

### Calculator
Input your formulas into this cell for analysis in the plots below

In [ ]:
def commonChannels(selected):
    global commonChannelsList
    commonChannelsList = []
    if (len(selected) == 1):
        commonChannelsList = readings[selected[0]].columns
    if (len(selected) > 1):
        commonChannelsList = readings[selected[0]].columns
        for s in list(selected):
            commonChannelsList = list(set(commonChannelsList) & set(readings[s].columns))
    _Aterm.options = list(commonChannelsList)
    _Aterm.source = selected
    _Bterm.options = list(commonChannelsList)
    _Bterm.source = selected
    _Cterm.options = list(commonChannelsList)
    _Cterm.source = selected
    _Dterm.options = list(commonChannelsList)
    _Dterm.source = selected
        
display(widgets.HTML('<hr><h4>Select the Files for your formulas to apply</h4>'))

selected=tuple()
def selectedFilesChannels(x):
    global selected
    selected = list(x)
    commonChannels(selected)
    
def calculateFormula(b):
    clear_output()
    A = _Aterm.value
    B = _Bterm.value
    C = _Cterm.value
    D = _Dterm.value
    Name = _formulaName.value
    for s in list(selected):
        result = functionFormula(s,A,B,C,D)
        readings[s][Name] = result
    print "Formula Added!"

def functionFormula(s, Aname, Bname, Cname, Dname): 
    calcData = pd.DataFrame()
    mergeData = pd.merge(pd.merge(pd.merge(readings[s].loc[:,(Aname,)],readings[s].loc[:,(Bname,)],left_index=True, right_index=True), readings[s].loc[:,(Cname,)], left_index=True, right_index=True),readings[s].loc[:,(Dname,)],left_index=True, right_index=True)
    calcData[Aname] = mergeData.iloc[:,0] #A
    calcData[Bname] = mergeData.iloc[:,1] #B
    calcData[Cname] = mergeData.iloc[:,2] #C
    calcData[Dname] = mergeData.iloc[:,3] #D
    A = calcData[Aname]
    B = calcData[Bname]
    C = calcData[Cname]
    D = calcData[Dname]
    result = eval(_formula.value)
    return result
            
# Nice formula to check increasing or decreasing signals: greater(np.gradient(smooth(A,10)),0)

layout = widgets.Layout(width='400px')
_Aterm = widgets.Dropdown(description = 'A', layout=layout)
_Bterm = widgets.Dropdown(description = 'B', layout=layout)
_Cterm = widgets.Dropdown(description = 'C', layout=layout)
_Dterm = widgets.Dropdown(description = 'D', layout=layout)

interact(selectedFilesChannels,x = widgets.SelectMultiple(options=readings.keys(), description=' ', selected_labels = selected,layout=widgets.Layout(width='700px')))

_formulaName = widgets.Text(description = 'Name: ')
_formula = widgets.Text(description = '=')
_ABtermsBox = widgets.HBox([_Aterm, _Bterm])
_CDtermsBox = widgets.HBox([_Cterm, _Dterm])
_termsBox = widgets.VBox([_ABtermsBox, _CDtermsBox])
_calculate = widgets.Button(description='Calculate')
_calculateBox = widgets.HBox([_formulaName,_formula, _calculate])
_calculate.on_click(calculateFormula)
display(widgets.HTML('<h4>Select the terms you wish to use in your formula </h4>'))
display(_termsBox)
display(widgets.HTML('<h4>Input your formula Below</h4>'))
display(_calculateBox)

## AlphaSense Baseline Calibration

### Parameters

In [27]:
# AlphaDelta PCB factor
factorPCB = 6.36

# Background Concentration (model assumption) - (from Masbit et al.)
backgroundConc = 5

### Functions

In [ ]:
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots
import plotly as ply                             # even more plots
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Layout

import pandas as pd
import numpy as np

import datetime
from scipy.stats.stats import linregress   
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')
from dateutil import relativedelta
from scipy.optimize import curve_fit



def ExtractBaseline(_data, _delta):
    '''
        Input:
            _data: dataframe containing signal to be baselined and index
            _delta : float for delta time (N periods)
        Output:
            result: vector containing baselined values
    ''' 
    
    result = np.zeros(len(_data))
    name = []
    for n in range(0, len(_data)):
        minIndex = max(n-_delta,0)
        maxIndex = min(n+_delta, len(_data))
        
        chunk = _data.values[minIndex:maxIndex]
        result[n] = min(chunk)

    return result

def findMax(_listF):
    '''
        Input: list to obtain maximum value
        Output: value and index of maximum in the list
    '''
    
    valMax=max(_listF)
    indexMax = _listF.index(max(_listF))
    # print 'Max Value found at index: {}, with value: {}'.format(indexMax, valMax)
    
    return valMax, indexMax

def exponential_smoothing(series, alpha):
    """
        series - dataset with timestamps
        alpha - float [0.0, 1.0], smoothing parameter
    """
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

def exponential_func(x, a, b, c):
     return a * np.exp(b * x) + c

def createBaselines(_dataBaseline, _dataCorr, _numberDeltas, _type_regress = 'linear', _plots = False, _verbose = False):
    '''
        Input:
            _dataBaseline: dataframe containing signal and index to be baselined
            _dataCorr: baseline data for regression
            _type_regress= 'linear', 'exponential', 'best' (based on p_value of both)
            _numberDeltas : vector of floats for deltas (N periods)
            _plots:  display plots or not
            _verbose: print info or not
            _type_regress: regression type (linear, exp, ... )
        Output:
            baseline: pandas dataframe baseline
        TODO:
            implement other types of regression
    '''
    
    resultData = _dataBaseline.copy()
    vectorCorr = _dataCorr.values

    name = resultData.name
    pearsons =[]
    
    for delta in _numberDeltas:
        resultData[(name +'_' +str(delta))] = ExtractBaseline(_dataBaseline, delta)
        slope, intercept, r_value, p_value, std_err = linregress(np.transpose(resultData[(name +'_' +str(delta))]), np.transpose(vectorCorr))
        pearsons.append(r_value)
    
    ## Find Max in the pearsons
    valMax, indexMax = findMax(pearsons)
        
    ## Find regression between _dataCorr
    baseline = pd.DataFrame(index = _dataBaseline.index)
    if _type_regress == 'linear':
        ## Fit with y = A + Bx
        slope, intercept, r_value, p_value, std_err = linregress(np.transpose(vectorCorr),resultData[(name + '_'+str(_numberDeltas[indexMax]))])
        baseline[(name + '_' + 'baseline_' +  _type_regress)] = intercept + slope*vectorCorr
    elif _type_regress == 'exponential':
        ## Fit with y = Ae^(Bx) -> logy = logA + Bx
        logy = np.log(resultData[(name + '_'+str(_numberDeltas[indexMax]))])
        slope, intercept, r_value, p_value, std_err = linregress(np.transpose(vectorCorr), logy)
        baseline[(name + '_' + 'baseline_' +  _type_regress)] = exponential_func(np.transpose(vectorCorr), np.exp(intercept), slope, 0)
    elif _type_regress == 'best':
        ## Find linear r_value
        slope_lin, intercept_lin, r_value_lin, p_value_lin, std_err_lin = linregress(np.transpose(vectorCorr),resultData[(name + '_'+str(_numberDeltas[indexMax]))])
        
        ## Find Exponential r_value
        logy = np.log(resultData[(name + '_'+str(_numberDeltas[indexMax]))])
        slope_exp, intercept_exp, r_value_exp, p_value_exp, std_err_exp = linregress(np.transpose(vectorCorr), logy)
        
        ## Pick which one is best
        if r_value_lin > r_value_exp:
            if _verbose:
                print 'Using linear regression'
            baseline[(name + '_' + 'baseline_' +  _type_regress)] = intercept_lin + slope_lin*vectorCorr
        else:
            if _verbose:
                print 'Using exponential regression'
            baseline[(name + '_' + 'baseline_' +  _type_regress)] = exponential_func(np.transpose(vectorCorr), np.exp(intercept_exp), slope_exp, 0)
            
    if _plots == True:
        with plt.style.context('seaborn-white'):
            fig1, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20,8))
            
            ax1.plot(_dataCorr.values, resultData[(name + '_'+str(_numberDeltas[indexMax]))], label = 'Baseline', linestyle='-', linewidth=0, marker='o')
            ax1.plot(_dataCorr.values, baseline[(name + '_' + 'baseline_' +  _type_regress)] , label = 'Regressed value', linestyle='-', linewidth=1, marker=None)
            legend = ax1.legend(loc='best')
            ax1.set_xlabel(_dataCorr.name)
            ax1.set_ylabel('Regression values')
            ax1.grid(True)
            
            ax2.plot(_dataBaseline.index, _dataBaseline.values, label = "Actual", linestyle=':', linewidth=1, marker=None)
            #[ax2.plot(resultData.index, resultData[(name +'_' +str(delta))].values, label="Delta {}".format(delta), marker=None,  linestyle='-', linewidth=1) for delta in _numberDeltas]
            ax2.plot(baseline.index, baseline.values, label='Baseline', marker = None)

            ax2.axis('tight')
            ax2.legend(loc='best')
            ax2.set_title("Baseline Extraction")
            ax2.grid(True)
            
            ax22 = ax2.twinx()
            ax22.plot(_dataCorr.index, _dataCorr.values, color = 'red', label = _dataCorr.name, linestyle='-', linewidth=1, marker=None)
            ax22.set_ylabel(_dataCorr.name, color = 'red')
            ax22.tick_params(axis='y', labelcolor='red')
            
            fig2, ax3 = plt.subplots(figsize=(20,8)) # two axes on figure
            ax3.plot(_numberDeltas, pearsons)
            ax3.axis('tight')
            ax3.set_title("R2 vs. Delta")
            ax3.set_xlabel('Delta')
            ax3.set_ylabel('R2')
            ax3.grid(True)

    return baseline, indexMax

def decompose(_data, plots = False):
    '''
            Function to decompose a signal into it's trend and normal variation
            Input:
                _data: signal to decompose
                plots: print plots or not (default False)
            Output:
                DataDecomp = _data - slope*_data.index
                slope, intercept = linear regression coefficients
    '''
    indexDecomp = np.arange(len(_data))

    slope, intercept, r_value, p_value, std_err = linregress(indexDecomp, np.transpose(_data.values))
    dataDecomp=pd.DataFrame(index = _data.index)
    name = _data.name
    result = []
    
    for n in range(len(_data)):
        result.append(float(_data.values[n]-slope*n))
    dataDecomp[(name + '_' + '_flat')] = result
    
    trend = slope*indexDecomp + intercept
    if plots == True:
        
        with plt.style.context('seaborn-white'):
            fig, ax = plt.subplots(figsize=(20,10))
            ax.plot(_data.index, _data.values, label = "Actual", marker = None)
            ax.plot(_data.index, dataDecomp[(name + '_' +'_flat')], marker = None, label = 'Flattened')
            ax.plot(_data.index, trend, label = 'Trend')
            ax.legend(loc="best")
            ax.axis('tight')
            ax.set_title("Signal Decomposition - "+ name)
            ax.set_xlabel('Index')
            ax.set_ylabel('Signal')
            ax.grid(True)
            
    return dataDecomp, slope, intercept

def calculateBaselineDay(_dataFrame, _listNames, _baselined, _baseliner, _type_regress, _trydecomp = False, _plots = False, _verbose = True):
    '''
        Function to calculate day-based baseline corrections
        Input:
            _dataFrame: pandas dataframe with datetime index containing 1 day of measurements
            _listNames: list containing column names of WE, AE, Temp and Hum
            _baselined: channel to calculate the baseline of
            _baseliner: channel to use as input for the baselined (baselined = f(baseliner)) 
            _type_regress: type of regression to perform (linear, exponential, best)
            _trydecomp: try trend decomposition (not needed . remove it)
            _plot: plot analytics or not
            _verbose: print analytics or not
        Output:
            _data_baseline: dataframe with baseline
            _baseline_corr: metadata containing analytics for long term analysis
    '''
    
    ## Un-pack list names
    alphaW, alphaA, temp, hum = _listNames
    
    ## Verify anticorrelation between temperature and humidity
    if _plots == True:
        with plt.style.context('seaborn-white'):
            fig2, (ax3, ax4) = plt.subplots(nrows = 2, ncols = 1,figsize=(20,10))
            ax3.scatter(_dataFrame[hum], _dataFrame[temp], marker = 'o', linewidth = 0)
            ax3.set_xlabel(_dataFrame[hum].name)
            ax3.set_ylabel(_dataFrame[temp].name)
            ax3.grid(True)
            
            colorH = 'red'
            colorT = 'blue'
            ax4.plot(_dataFrame.index, _dataFrame[hum], c = colorH, label = _dataFrame[hum].name, marker = None)
            ax5 = ax4.twinx()
            ax5.plot(_dataFrame.index, _dataFrame[temp], c = colorT, label = _dataFrame[temp].name, marker = None)
            ax4.tick_params(axis='y', labelcolor=colorH)
            ax5.tick_params(axis='y', labelcolor=colorT)
            ax4.set_xlabel('Time')
            ax4.set_ylabel(_dataFrame[temp].name, color = colorH)
            ax5.set_ylabel(_dataFrame[hum].name, color = colorT)
            ax4.grid(True)

    ## Correlation between temperature and working electrode raw
    slopenDC, interceptnDC, r_valuenDC, p_valuenDC, std_errnDC = linregress(np.transpose(_dataFrame[_baseliner].values), np.transpose(_dataFrame[_baselined].values))
   
    ## Create Baselines
    ## TODO Automate this
    deltas = np.arange(1,70,1)
    data_baseline, indexMax = createBaselines(_dataFrame[_baselined], _dataFrame[_baseliner], deltas, _type_regress, _plots, _verbose)

    if _trydecomp == True:
        dataDecomp = pd.DataFrame(index = _dataFrame.index)
        # Decompose Trend - Check if decomposition helps at all
        dataDecomp[alphaW], dataWSlope, dataWIntercept = decompose(_dataFrame[alphaW], _plots)
        dataDecomp[alphaA], dataASlope, dataAIntercept = decompose(_dataFrame[alphaA], _plots)
        dataDecomp[temp], dataCorrSlope, dataCorrIntercept = decompose(_dataFrame[temp], _plots)

        slopeDC, interceptDC, r_valueDC, p_valueDC, std_errDC = linregress(np.transpose(dataDecomp[_baseliner]), np.transpose(dataDecomp[_baselined]))
        data_baselineDecomp, indexMaxDecomp = createBaselines(dataDecomp[_baselined], dataDecomp[_baseliner], deltas, _type_regress, _plots)
        slopeBADecomp, interceptBADecomp, r_valueBADecomp, p_valuenBADecomp, std_errnBADecomp = linregress(np.transpose(data_baseline.values), np.transpose(_dataFrame[alphaA].values))

    ## Try to find a correlation with the auxiliary electrode
    ## Correlation between Baseline and original auxiliary
    slopeBA, interceptBA, r_valueBA, p_valueBA, std_errBA = linregress(np.transpose(data_baseline.values), np.transpose(_dataFrame[alphaA].values))
    
    deltaAuxBas = data_baseline.values-_dataFrame[alphaA].values
    ratioAuxBas = data_baseline.values/_dataFrame[alphaA].values
    
    deltaAuxBas_avg = np.mean(deltaAuxBas)
    ratioAuxBas_avg = np.mean(ratioAuxBas)
    
    # Pre filter the metadata
    if slopeBA > 0 and r_valueBA > 0.3:
        valid = True
    else:
        valid = False
    baselineCorr = (slopeBA, interceptBA, r_valueBA, p_valueBA, std_errBA, deltaAuxBas_avg, ratioAuxBas_avg, indexMax, valid)
    
    if _verbose == True:
        
        print '-------------------'
        print 'Correlation coeffs'
        print '-------------------'
        print 'Correlation coefficient without decomposed trend: {}'.format(r_valuenDC)
        if _trydecomp == True:
            print 'Correlation coefficient with decomposed trend: {}'.format(r_valueDC)
        
            print '-------------------'
            print 'Slopes'
            print '-------------------'
            print 'Slope Working Electrode: {} \n Slope Aux Electrode: {} \n Slope Temperature : {} \t Ratio WE/T: {}'.format(dataWSlope, dataASlope, dataCorrSlope, dataWSlope/dataCorrSlope)
    
        print '-------------------'
        print 'Auxiliary Electrode'
        print '-------------------'
        print 'Correlation coefficient of Baseline and Original auxiliary: {}'.format(r_valueBA)
        print 'Baseline Correlation Slope: {} \t Intercept: {}'.format(slopeBA, interceptBA)
        if _trydecomp == True:
            print 'Correlation coefficient of Baseline and Original auxiliary with Decomp: {}'.format(r_valueBADecomp)
            print 'Baseline Correlation Slope: {} \t Intercept with Decomp: {}'.format(slopeBADecomp, interceptBADecomp)
        
        print 'Average Delta: {} \t Average Ratio: {}'.format(deltaAuxBas_avg, ratioAuxBas_avg)

    if _plots == True:
        with plt.style.context('seaborn-white'):
            
            if _trydecomp == False:
                fig2, ax3 = plt.subplots(figsize=(20,8))
            else: 
                fig2, (ax3, ax4) = plt.subplots(nrows = 1, ncols = 2, figsize=(20,8))
                ax4.plot(data_baselineDecomp.index, data_baselineDecomp.values, label='Baseline', marker = None)
                ax4.plot(dataDecomp.index, dataDecomp[alphaW], label = 'Working Decomp', marker = None)
                ax4.plot(dataDecomp.index, dataDecomp[alphaA], label = 'Auxiliary Decomp', marker = None)
                ax4.legend(loc="best")
                ax4.axis('tight')
                ax4.set_title("Baseline Compensated")
                ax4.set(xlabel='Time', ylabel='Ouput-mV')
                ax4.grid(True)
                ax4.set_ylim(min(min(dataDecomp[temp]),min(data_baselineDecomp.values)) -5,max(max(dataDecomp[temp]),max(data_baselineDecomp.values))+5)
                
                ax6 = ax4.twinx()
                ax6.plot(dataDecomp.index, dataDecomp[temp], label='Temperature Decomp', c = 'red', marker = None)
                ax6.tick_params(axis='y', labelcolor ='red')
                ax6.set_ylabel('Temperature (degC)', color = 'red')
                ax6.set_ylim(min(min(dataDecomp[temp]),min(data_baselineDecomp.values)) -5,max(max(dataDecomp[temp]),max(data_baselineDecomp.values))+5)
            
            ax3.plot(data_baseline.index, data_baseline.values, label='Baseline', marker = None)
            ax3.plot(_dataFrame.index, _dataFrame[alphaW], label='Original Working', marker = None)
            ax3.plot(_dataFrame.index, _dataFrame[alphaA], label='Original Auxiliary', marker = None)

            ax3.legend(loc="best")
            ax3.axis('tight')
            ax3.set_title("Baseline Not Compensated")
            ax3.set(xlabel='Time', ylabel='Ouput-mV')
            ax3.grid(True)
            ax3.set_ylim(min(min(_dataFrame[temp]),min(data_baseline.values)) -5,max(max(_dataFrame[temp]),max(data_baseline.values))+5)
            
            if _trydecomp == True:
                ax5 = ax3.twinx()
                ax5.plot(dataDecomp.index, dataDecomp[temp], label='Temperature Decomp', c = 'red', marker = None)
                ax5.tick_params(axis='y', labelcolor ='red')
                ax5.set_ylabel(dataDecomp[temp].name, color = 'red')
                ax5.set_ylim(min(min(dataDecomp[temp]),min(data_baselineDecomp.values)) -5,max(max(dataDecomp[temp]),max(data_baselineDecomp.values))+5)
                
            fig3, ax7 = plt.subplots(figsize=(20,8))
            
            ax7.plot(_dataFrame[temp], _dataFrame[alphaW], label='W - Raw', marker='o',  linestyle=None, linewidth = 0)
            ax7.plot(_dataFrame[temp], _dataFrame[alphaA], label ='A - Raw', marker='v', linewidth=0)
            
            if _trydecomp == True:
                ax7.plot(dataDecomp[temp], dataDecomp[alphaA], label ='A - Trend Decomposed', marker='v', linewidth=0)
                ax7.plot(dataDecomp[temp], dataDecomp[alphaW], label = 'W - Trend Decomposed',marker='o', linestyle=None, linewidth = 0)
            
            ax7.legend(loc="best")
            ax7.axis('tight')
            ax7.set_title("Output vs. Temperature")
            ax7.set(xlabel='Temperature', ylabel='Ouput-mV')
            ax7.grid(True)
    
    return data_baseline, baselineCorr

def findDates(_dataframe):
    '''
        Input: pandas dataframe with datetime index
        Output: rounded up min day, floor max day and number of days between the min and max dates
    '''
    range_days = (_dataframe.index.max()-_dataframe.index.min()).days
    min_date_df = _dataframe.index.min().ceil('D')
    max_date_df = _dataframe.index.max().floor('D')
    
    return min_date_df, max_date_df, range_days

def calculatePollutant(_dataframe, _pollutant, _sensorID, _baselineType, _refAvail, _dataframeRef, _refName, _listNames, _overlapHours = 0, _type_regress = 'best', _filterExpSmoothing = 0.2, _trydecomp = False, _plotsInter = False, _plotResult = True, _verbose = False):
    
    ## TODO: read this from DB

    Sensitivity = alpha_calData.loc[_sensorID,'Sensitivity 1']
    nWA = alpha_calData.loc[_sensorID,'Zero Current']/alpha_calData.loc[_sensorID,'Aux Zero Current']

    dataframeResult = dataframe.copy()
    alphaW, alphaA, temp, hum = _listNames
    
    if _baselineType == 'single_temp':
        baseliner = temp
    elif _baselineType == 'single_hum':
        baseliner = hum
    elif _baselineType == 'single_aux':
        baseliner = alphaA
    baselined = alphaW
    
    
    if _pollutant == 'NO2':
        molecularWeight = 46
    elif _pollutant == 'CO':
        molecularWeight = 28
    elif _pollutant == 'O3':
        molecularWeight = 48
    else:
        print 'Not Supported pollutant'
        return
    
    # Temporary for comparison (just name it _pollutant in the future)
    pollutant_column = (_pollutant + '_' + _baselineType)
        
    ## Find min, max and range of days
    min_date_df, max_date_df, range_days = findDates(_dataframe)
    print 'Data Range from {} to {} with {} days'.format(min_date_df, max_date_df, range_days)
    
    ## TODO: add metrics in dataframe calibration
    #dataframeCalibration = pd.DataFrame(index = dataframe.index)
    #dataframeCalibration = dataframeCalibration.groupby(pd.TimeGrouper(freq='day')).aggregate(np.mean)
    
    for i in range(range_days):
    
        min_date_ovl = max(_dataframe.index.min(), (min_date_df + pd.DateOffset(days=i) - pd.DateOffset(hours = _overlapHours)))
        max_date_ovl = min(_dataframe.index.max(), (min_date_ovl + pd.DateOffset(days=1) + pd.DateOffset(hours = _overlapHours + relativedelta.relativedelta(min_date_df + pd.DateOffset(days=i),min_date_ovl).hours)))
        
        min_date_novl = max(min_date_df, (min_date_df + pd.DateOffset(days=i)))
        max_date_novl = min(max_date_df, (min_date_novl + pd.DateOffset(days=1)))
        
        if _verbose:
            print '------------------------------------------------------------------'
            print 'Calculating day {}, with range: {} \t to {}'.format(i, min_date_ovl, max_date_ovl)
            print '------------------------------------------------------------------'
        
        dataframeTrim = _dataframe[_dataframe.index > min_date_ovl]
        dataframeTrim = dataframeTrim[dataframeTrim.index <= max_date_ovl]
        
        if i == 0:
                CorrParams = list()
                
        if dataframeTrim.empty:
            if _verbose:
                print 'No data between these dates'
                
            nanV =np.ones(9)
            nanV.fill(np.nan)
            
            CorrParams.append(nanV)
            
        else:
            
            dataframeTrim[alphaW + '_baseline'], CorrParamsTrim = calculateBaselineDay(dataframeTrim, _listNames, baselined, baseliner, _type_regress, _trydecomp, _plotsInter, _verbose)
            
            dataframeTrim = dataframeTrim[dataframeTrim.index > min_date_novl]
            dataframeTrim = dataframeTrim[dataframeTrim.index <= max_date_novl]
            
            if _pollutant == 'CO' or _pollutant == 'NO2':
                dataframeTrim[pollutant_column] = backgroundConc + factorPCB*(dataframeTrim[alphaW] - dataframeTrim[alphaW + '_baseline'])/Sensitivity*1000
            #elif _pollutant == 'O3':
                #dataframeTrim[pollutant_column]
            
            dataframeResult = dataframeResult.combine_first(dataframeTrim)
            CorrParams.append(CorrParamsTrim)
    
    dataframeResult[pollutant_column + '_filter'] = exponential_smoothing(dataframeResult[pollutant_column].fillna(0), filterExpSmoothing)
                
    ## Retrieve metadata
    labelsCP = ['slopeBA', 'interceptBA', 'r_valueBA', 'p_valueBA', 'std_errBA', 'deltaAuxBas_avg', 'ratioAuxBas_avg', 'indexMax', 'valid']
    CorrParamsDF = pd.DataFrame(CorrParams, columns = labelsCP, index = [(min_date_df+ pd.DateOffset(days=i)).strftime('%Y-%m-%d') for i in range(range_days)])
    
    ## Find average ratio for hole dataset(maybe TODO: day by day)
    deltaAuxBas_avg = CorrParamsDF.loc[CorrParamsDF['valid'].fillna(False), 'deltaAuxBas_avg'].mean(skipna = True)
    deltaAuxBas_std = CorrParamsDF.loc[CorrParamsDF['valid'].fillna(False), 'deltaAuxBas_avg'].std(skipna = True)
    ratioAuxBas_avg = CorrParamsDF.loc[CorrParamsDF['valid'].fillna(False), 'ratioAuxBas_avg'].mean(skipna = True)
    ratioAuxBas_std = CorrParamsDF.loc[CorrParamsDF['valid'].fillna(False), 'ratioAuxBas_avg'].std(skipna = True)
    
    ## TODO - Make the check for outliers and mark them out
    # CorrParamsDF['valid'] = deltaAuxBas_avg-deltaAuxBas_std <= CorrParamsDF['deltaAuxBas_avg'] <= deltaAuxBas_avg-deltaAuxBas_std
    # CorrParamsDF['valid'] = ratioAuxBas_avg-ratioAuxBas_std <= CorrParamsDF['ratioAuxBas_avg'] <= ratioAuxBas_avg-ratioAuxBas_std
    # 
    # print CorrParamsDF
    # 
    # deltaAuxBas_avg = CorrParamsDF.loc[CorrParamsDF['valid'].fillna(False), 'deltaAuxBas_avg'].mean(skipna = True)
    # deltaAuxBas_std = CorrParamsDF.loc[CorrParamsDF['valid'].fillna(False), 'deltaAuxBas_avg'].std(skipna = True)
    # ratioAuxBas_avg = CorrParamsDF.loc[CorrParamsDF['valid'].fillna(False), 'ratioAuxBas_avg'].mean(skipna = True)
    # ratioAuxBas_std = CorrParamsDF.loc[CorrParamsDF['valid'].fillna(False), 'ratioAuxBas_avg'].std(skipna = True)
    
    if _verbose:
        
        print '------------------------'
        print ' Meta Data'
        print '------------------------'
        print CorrParamsDF
        
        print '------------------------'
        print 'Average Delta between baseline and auxiliary electrode: {}, and ratio {}:'.format(deltaAuxBas_avg, ratioAuxBas_avg)
        print 'Std Dev of Delta between baseline and auxiliary electrode: {}, and ratio {}:'.format(deltaAuxBas_std, ratioAuxBas_std)
        print '------------------------'
    
    if _plotResult:
        
        fig1 = tls.make_subplots(rows=3, cols=1, shared_xaxes=True, print_grid=False)
        
        fig1.append_trace({'x': dataframeResult.index, 'y': dataframeResult[alphaW], 'type': 'scatter', 'line': dict(width = 2), 'name': dataframeResult[alphaW].name}, 1, 1)
        fig1.append_trace({'x': dataframeResult.index, 'y': dataframeResult[alphaA], 'type': 'scatter', 'line': dict(width = 2), 'name': dataframeResult[alphaA].name}, 1, 1)
        fig1.append_trace({'x': dataframeResult.index, 'y': dataframeResult[alphaA] * nWA, 'type': 'scatter', 'line': dict(width = 1, dash = 'dot'), 'name': 'AuxCor Alphasense'}, 1, 1)
        fig1.append_trace({'x': dataframeResult.index, 'y': dataframeResult[alphaW + '_baseline'], 'type': 'scatter', 'line': dict(width = 2), 'name': 'Baseline'}, 1, 1)
        
        fig1.append_trace({'x': dataframeResult.index, 'y': dataframeResult[pollutant_column], 'type': 'scatter', 'line': dict(width = 1, dash = 'dot'), 'name': dataframeResult[pollutant_column].name}, 2, 1)
        fig1.append_trace({'x': dataframeResult.index, 'y': dataframeResult[pollutant_column + '_filter'], 'type': 'scatter', 'name': (dataframeResult[pollutant_column + '_filter'].name)}, 2, 1)
        
        if _refAvail:
            fig1.append_trace({'x': dataframeRef.index, 'y': dataframeRef[_refName]*(24.45/molecularWeight), 'type': 'scatter', 'name': dataframeRef[_refName].name}, 2, 1)
            
        fig1.append_trace({'x': dataframeResult.index, 'y': dataframeResult[temp], 'type': 'scatter', 'line': dict(width = 1, dash = 'dot'), 'name': dataframeResult[temp].name}, 3, 1)
        fig1.append_trace({'x': dataframeResult.index, 'y': dataframeResult[hum], 'type': 'scatter', 'name': (dataframeResult[hum].name)}, 3, 1)
        
        fig1['layout'].update(height = 800, legend=dict(x=-.1, y=5), xaxis=dict(title='Time'), title = 'Baseline Correction for {}'.format(_pollutant))
                               
        ply.offline.iplot(fig1)
    
    return dataframeResult, CorrParamsDF

### Correction

In [29]:
# TODO: convert to auto
#alphaW = 'AlphaDelta 2W-mV'
#alphaA = 'AlphaDelta 2A-mV'
#temp = 'AlphaDelta Temperature-C'
#hum = 'AlphaDelta Humidity-%'
alphaW = 'AlphaDelta 2W'
alphaA = 'AlphaDelta 2A'
temp = 'AlphaDelta Temperature-C'
hum = 'AlphaDelta Humidity-%'

sensorID = 162581712
Sensitivity = alpha_calData.loc[sensorID,'Sensitivity 1']
nWA = alpha_calData.loc[sensorID,'Zero Current']/alpha_calData.loc[sensorID,'Aux Zero Current']


listNames = (alphaW, alphaA, temp, hum)

## Hyperparameters
overlapHours = 2 # Overlap in hours for each day (index = [day(i)-overlapHours, day(i+1)+overlapHours])
filterExpSmoothing = 0.2

## TODO, get this from widgets?
## Select data

#dataframe = readings['MA04_Formulae_2.csv'].dropna()
dataframe = readings['POST001_Alpha_sen_B.CSV'].dropna()

dataframeRef = readings['CaARPAE_MAData_2_COR.csv'].dropna()
refName = 'NO2 ug/m3'

dataframe, CorrParams = calculatePollutant(_dataframe = dataframe, 
                        _pollutant = 'NO2', 
                        _sensorID = sensorID, 
                        _baselineType = 'single_temp',
                        _refAvail = False, 
                        _dataframeRef = dataframeRef, 
                        _refName = refName, 
                        _listNames = listNames, 
                        _overlapHours = overlapHours, 
                        _type_regress = 'best', 
                        _filterExpSmoothing = filterExpSmoothing, 
                        _trydecomp = False,
                        _plotsInter = False, 
                        _plotResult = False,
                        _verbose = False)

dataframe, CorrParams = calculatePollutant(_dataframe = dataframe, 
                        _pollutant = 'NO2', 
                        _sensorID = '123456', 
                        _baselineType = 'single_hum', 
                        _refAvail = False, 
                        _dataframeRef = dataframeRef, 
                        _refName = refName, 
                        _listNames = listNames, 
                        _overlapHours = overlapHours, 
                        _type_regress = 'best', 
                        _filterExpSmoothing = filterExpSmoothing, 
                        _trydecomp = False,
                        _plotsInter = False, 
                        _plotResult = False,
                        _verbose = False)

dataframe, CorrParams = calculatePollutant(_dataframe = dataframe, 
                        _pollutant = 'NO2', 
                        _sensorID = '123456', 
                        _baselineType = 'single_aux', 
                        _refAvail = False, 
                        _dataframeRef = dataframeRef, 
                        _refName = refName, 
                        _listNames = listNames, 
                        _overlapHours = overlapHours, 
                        _type_regress = 'best', 
                        _filterExpSmoothing = filterExpSmoothing, 
                        _trydecomp = False,
                        _plotsInter = False, 
                        _plotResult = False,
                        _verbose = False)

fig1 = tls.make_subplots(rows=4, cols=1, shared_xaxes=True, print_grid=False)
        
fig1.append_trace({'x': dataframe.index, 'y': dataframe[alphaW], 'type': 'scatter', 'line': dict(width = 2), 'name': dataframe[alphaW].name}, 1, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe[alphaA], 'type': 'scatter', 'line': dict(width = 2), 'name': dataframe[alphaA].name}, 1, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe[alphaA] * nWA, 'type': 'scatter', 'line': dict(width = 1, dash = 'dot'), 'name': 'AuxCor Alphasense'}, 1, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe[alphaW + '_baseline'], 'type': 'scatter', 'line': dict(width = 2), 'name': 'Baseline'}, 1, 1)
        
fig1.append_trace({'x': dataframe.index, 'y': dataframe['NO2_single_aux'], 'type': 'scatter', 'line': dict(width = 1, dash = 'dot'), 'name': dataframe['NO2_single_aux'].name}, 2, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe['NO2_single_aux' + '_filter'], 'type': 'scatter', 'name': (dataframe['NO2_single_aux' + '_filter'].name)}, 2, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe['NO2_single_temp'], 'type': 'scatter', 'line': dict(width = 1, dash = 'dot'), 'name': dataframe['NO2_single_temp'].name}, 2, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe['NO2_single_temp' + '_filter'], 'type': 'scatter', 'name': (dataframe['NO2_single_temp' + '_filter'].name)}, 2, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe['NO2_single_hum'], 'type': 'scatter', 'line': dict(width = 1, dash = 'dot'), 'name': dataframe['NO2_single_hum'].name}, 2, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe['NO2_single_hum' + '_filter'], 'type': 'scatter', 'name': (dataframe['NO2_single_hum' + '_filter'].name)}, 2, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe[temp], 'type': 'scatter', 'name': dataframe[temp].name, 'yaxis' : 'y2'}, 3, 1)
fig1.append_trace({'x': dataframe.index, 'y': dataframe[hum], 'type': 'scatter', 'name': dataframe[hum].name}, 4, 1)         

fig1.append_trace({'x': dataframeRef.index, 'y': dataframeRef[refName]*(24.45/molecularWeight), 'type': 'scatter', 'name': dataframeRef[refName].name}, 2, 1)
            
fig1['layout'].update(height = 1000, 
                      legend=dict(x=-.1, y=5), 
                      xaxis=dict(title='Time'), 
                      yaxis1 = dict(title='Sensor Output - mV'), 
                      yaxis2 = dict(title='Pollutant - ppm'),
                      yaxis3 = dict(title='Temperature - degC'),
                      yaxis4 = dict(title='Humidity - %'),
                      title = 'Baseline Correction for {}'.format('NO2'))
            
ply.offline.iplot(fig1)

NameError: name 'readings' is not defined

In [ ]:
with plt.style.context('seaborn-white'):

    fig2, ax2 = plt.subplots(figsize=(20,10))
    ax2.scatter(dataW_baseline.values, dataA.values, c = dataCorrT, cmap = 'autumn', label='Original Auxiliary', marker = 'o', linewidth = 0) 
    ax2.scatter(dataW_baseline.values, dataADecomp.values, c = dataCorrT, cmap = 'winter', label='Decomp Auxiliary', marker = 'o', linewidth = 0) 
    legend = ax2.legend(loc="best")
    legend.legendHandles[0].set_color(plt.cm.autumn(.8))
    legend.legendHandles[1].set_color(plt.cm.winter(.8))
    #legend.legendHandles[2].set_color(plt.cm.Reds(.8))
    ax2.axis('tight')
    ax2.set_title("Auxiliary vs. Baseline")
    ax2.set_xlabel('Baseline -mV')
    ax2.set_ylabel('Auxiliary electrode output -mV')
    ax2.grid(True)
    
    fig3, (ax3, ax4) = plt.subplots(nrows = 1, ncols = 2, figsize=(20,8))
    ax3.scatter(dataW.index, delta, c = dataCorrT.values, cmap = 'autumn', label = 'Delta Decomp', marker = None)
    ax4.scatter(dataW.index, ratio, c = dataCorrT.values, cmap = 'autumn', label = 'Ratio Baseline/Aux Decomp', marker = None)
    ax3.legend(loc="best")
    ax3.axis('tight')
    ax3.set_title("Delta Baseline - Auxiliary electrode")
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Electrode output -mV')
    ax3.set_xlim(min_date, max_date)
    
    ax3.grid(True)
    ax4.legend(loc="best")
    ax4.axis('tight')
    ax4.set_title("Ratio Baseline / Auxiliary electrode")
    ax4.set_xlabel('Time')
    ax4.set_ylabel('Electrode output -mV')
    ax4.grid(True)
    ax4.set_xlim(min_date, max_date)

## Data Export

### Local Data Export
Export data to csv with formulas

In [ ]:
global selected
selected = []

def selectedFilesChannels(x):
    selected = list(x)
    
def exportFile(b):
    for i in range(len(selected)):
        b.f = selected[i]
        exportDir = exportPath.value
        if not os.path.exists(exportDir): os.mkdir(exportDir)
        savePath = os.path.join(exportDir, b.f)
        if not os.path.exists(savePath):
            readings[b.f].to_csv(savePath, sep=",")
            display(FileLink(savePath))
        else:
            display(widgets.HTML(' File Already exists!'))

display(widgets.HTML('<h3>Export Files</h3>'))
exportPath = widgets.Text(description = 'Type in export path  ', layout=widgets.Layout(width='700px'))
eb = widgets.Button(description='Export file', layout=widgets.Layout(width='150px'))
eb.on_click(exportFile)

interact(selectedFilesChannels,x = widgets.SelectMultiple(options=readings.keys(), description='Select multiple files', selected_labels = selected,layout=widgets.Layout(width='700px')))
exportBox = widgets.HBox([exportPath,eb])
display(exportBox)

---
## Exploratory Data Analysis

### Time Series Plots

In [ ]:
# Plot Y limits
setLimits = False
maxY = 15000
minY = 0

toshow = []
axisshow = []
# meanTable = []

def show_sensors(Source):
    _sensor_drop.options = [s for s in list(readings[Source].columns)]
    _sensor_drop.source = Source
    _min_date.value = readings[Source].index.min()._short_repr
    _max_date.value = readings[Source].index.max()._short_repr

def clear_all(b):
    clear_output()
    del toshow[:]
    del axisshow[:]

def add_sensor(b):
    clear_output()
    d = [_sensor_drop.source, _sensor_drop.value]
    
    if d not in toshow: 
        toshow.append(d)
        axisshow.append(_axis_drop.value)
        
    plot_data = readings[toshow[0][0]].loc[:,(toshow[0][1],)]
    list_data_primary = []
    list_data_secondary = []
    list_data_terciary = []
    
    if b.slice_time:
        plot_data = plot_data[plot_data.index > _min_date.value]
        plot_data = plot_data[plot_data.index < _max_date.value]
    
    if len(toshow) > 1:
        for i in range(1, len(toshow)):
            plot_data = pd.merge(plot_data, readings[toshow[i][0]].loc[:,(toshow[i][1],)], left_index=True, right_index=True)

    print '-------------------------------------'
    print ' Medias:\n'
    meanTable = []
    for d in toshow:
        myMean = ' ' + d[0]  + "\t" + d[1] + "\t"
        meanTable.append(myMean)   
    res = plot_data.mean()
    for i in range(len(meanTable)): print meanTable[i] + '%.2f' % (res[i])
    print '-------------------------------------'

    # Change columns naming
    changed = []
    for i in range(len(plot_data.columns)):
        changed.append(toshow[i][0] + ' - '+ plot_data.columns[i])
    plot_data.columns = changed
    
    subplot_rows = 0
    if len(toshow) > 0:
        for i in range(len(toshow)):
            if axisshow[i]=='1': 
                list_data_primary.append(str(changed[i]))
                subplot_rows = max(subplot_rows,1)
            if axisshow[i]=='2': 
                list_data_secondary.append(str(changed[i]))
                subplot_rows = max(subplot_rows,2)
            if axisshow[i]=='3': 
                list_data_terciary.append(str(changed[i]))
                subplot_rows = max(subplot_rows,3)
          
        
    fig1 = tls.make_subplots(rows=subplot_rows, cols=1, shared_xaxes=_synchroniseXaxis.value)

    #if len(list_data_primary)>0:
        #fig1 = plot_data.iplot(kind='scatter', y = list_data_primary, asFigure=True, layout = layout)
    #ply.offline.iplot(fig1)
    
    for i in range(len(list_data_primary)):
        fig1.append_trace({'x': plot_data.index, 'y': plot_data[list_data_primary[i]], 'type': 'scatter', 'name': list_data_primary[i]}, 1, 1)

    for i in range(len(list_data_secondary)):
        fig1.append_trace({'x': plot_data.index, 'y': plot_data[list_data_secondary[i]], 'type': 'scatter', 'name': list_data_secondary[i]}, 2, 1)
    
    for i in range(len(list_data_terciary)):
        fig1.append_trace({'x': plot_data.index, 'y': plot_data[list_data_terciary[i]], 'type': 'scatter', 'name': list_data_terciary[i]}, 3, 1)

    if setLimits: 
        fig1['layout'].update(height = 600,
                            legend=dict(x=-.1, y=5) ,
                           xaxis=dict(title='Time'))
                          
    else:
        fig1['layout'].update(height = 600,
                              legend=dict(x=-.1, y=5) ,
                           xaxis=dict(title='Time'))
                           
    ply.offline.iplot(fig1)
            
def reset_time(b):
    _min_date.value = readings[b.src.value].index.min()._short_repr
    _max_date.value = readings[b.src.value].index.max()._short_repr

layout=widgets.Layout(width='330px')
_kit = widgets.Dropdown(options=[k for k in readings.keys()], layout=layout)
_kit_drop = widgets.interactive(show_sensors, Source=_kit, layout=layout)

_sensor_drop = widgets.Dropdown(layout=layout)
_b_add = widgets.Button(description='Add to Plot', layout=widgets.Layout(width='120px'))
_b_add.on_click(add_sensor)
_b_add.slice_time = False
_b_reset_all = widgets.Button(description='Clear all', layout=widgets.Layout(width='120px'))
_b_reset_all.on_click(clear_all)

_axis_drop = widgets.Dropdown(
    options=['1', '2', '3'],
    value='1',
    description='Subplot:',
    disabled=False,
)

_synchroniseXaxis = widgets.Checkbox(value=False, description='Synchronise X axis', disabled=False, layout=widgets.Layout(width='300px'))
_min_date = widgets.Text(description='Start date:', layout=widgets.Layout(width='330px'))
_max_date = widgets.Text(description='End date:', layout=widgets.Layout(width='330px'))
_b_apply_time = _b_reset = widgets.Button(description='Apply dates', layout=widgets.Layout(width='100px'))
_b_apply_time.on_click(add_sensor)
_b_apply_time.slice_time = True
_b_reset_time = _b_reset = widgets.Button(description='Reset dates', layout=widgets.Layout(width='100px'))
_b_reset_time.on_click(reset_time)
_b_reset_time.src = _kit

_sensor_box = widgets.HBox([_kit_drop, _sensor_drop])
_plot_box = widgets.HBox([_axis_drop, _synchroniseXaxis, _b_add , _b_reset_all])
_time_box = widgets.HBox([_min_date,_max_date, _b_reset_time, _b_apply_time])
_root_box = widgets.VBox([_time_box, _sensor_box, _plot_box])
display(_root_box)

### TODO: Basic Sensor Correlations

#### Full Seaborn Correlogram

In [ ]:
def paint(Source):
    clear_output()
    sns.set(font_scale=1.4)
    g = sns.PairGrid(readings.values()[0])
    g = g.map(plt.scatter)

_kit = widgets.Dropdown(options=[k for k in readings.keys()], layout=layout)
_kit_drop = widgets.interactive(paint, Source=_kit, layout=layout)
display(_kit_drop)

#### Basic Seaborn XYPlot

In [ ]:
cropTime = False
min_date = "2001-01-01 00:00:01"
max_date = "2001-01-01 00:00:01"
doubleAxis = True

def show_sensors_A(Source):
    A_sensors_drop.options = [s for s in list(readings[Source].columns)]
    A_sensors_drop.source = Source
    minCropDate.value = readings[Source].index.min()._short_repr
    maxCropDate.value = readings[Source].index.max()._short_repr
    
def show_sensors_B(Source):
    B_sensors_drop.options = [s for s in list(readings[Source].columns)]
    B_sensors_drop.source = Source
    minCropDate.value = readings[Source].index.min()._short_repr
    maxCropDate.value = readings[Source].index.max()._short_repr
    
def redraw(b):
    cropTime = cropTimeCheck.value
    doubleAxis = doubleAxisCheck.value
    min_date = minCropDate.value
    max_date = maxCropDate.value
    mergedData = pd.merge(readings[A_kit.value].loc[:,(A_sensors_drop.value,)], readings[B_kit.value].loc[:,(B_sensors_drop.value,)], left_index=True, right_index=True, suffixes=('_'+A_kit.value, '_'+B_kit.value))
    clear_output()
    
    if cropTime:
        mergedData = mergedData[mergedData.index > min_date]
        mergedData = mergedData[mergedData.index < max_date]
        
    #jointplot
    df = pd.DataFrame()
    A = A_sensors_drop.value + '-' + A_kit.value
    B = B_sensors_drop.value + '-' + B_kit.value
    df[A] = mergedData.iloc[:,0]
    df[B] = mergedData.iloc[:,1]
    
    sns.set(font_scale=1.3)
    sns.jointplot(A, B, data=df, kind="reg", color="b", size=12, scatter_kws={"s": 80});
    print "data from " + str(df.index.min()) + " to " + str(df.index.max())                      
    pearsonCorr = list(df.corr('pearson')[list(df.columns)[0]])[-1]
    print 'Pearson correlation coefficient: ' + str(pearsonCorr)
    print 'Coefficient of determination R²: ' + str(pearsonCorr*pearsonCorr)

    if cropTime: 
        
        if (doubleAxis):
            layout = go.Layout(
                legend=dict(x=-.1, y=1.2), 
                xaxis=dict(range=[min_date, max_date],title='Time'), 
                yaxis=dict(zeroline=True, title=A, titlefont=dict(color='rgb(0,97,255)'), tickfont=dict(color='rgb(0,97,255)')),
                yaxis2=dict(title=B,titlefont=dict(color='rgb(255,165,0)'), tickfont=dict(color='rgb(255,165,0)'), overlaying='y', side='right')
            )
        else:
            layout = go.Layout(
                legend=dict(x=-.1, y=1.2), 
                xaxis=dict(range=[min_date, max_date],title='Time'), 
                yaxis=dict(zeroline=True, title=A, titlefont=dict(color='rgb(0,97,255)'), tickfont=dict(color='rgb(0,97,255)')),
            )
            
    else:
        if (doubleAxis):
            layout = go.Layout(
            legend=dict(x=-.1, y=1.2), 
            xaxis=dict(title='Time'), 
            yaxis=dict(title=A, titlefont=dict(color='rgb(0,97,255)'), tickfont=dict(color='rgb(0,97,255)')),
            yaxis2=dict(title=B, titlefont=dict(color='rgb(255,165,0)'), tickfont=dict(color='rgb(255,165,0)'), overlaying='y', side='right')
            )
        else:
            layout = go.Layout(
            legend=dict(x=-.1, y=1.2), 
            xaxis=dict(title='Time'), 
            yaxis=dict(zeroline=True, title=A, titlefont=dict(color='rgb(0,97,255)'), tickfont=dict(color='rgb(0,97,255)')),
            )
        
    trace0 = go.Scatter(x=df[A].index, y=df[A], name = A,line = dict(color='rgb(0,97,255)'))
    
    if (doubleAxis):
        trace1 = go.Scatter(x=df[B].index,y=df[B],name=B, yaxis='y2', line = dict(color='rgb(255,165,0)'))
    else:
        trace1 = go.Scatter(x=df[B].index,y=df[B],name=B, line = dict(color='rgb(255,165,0)'))
    data = [trace0, trace1]
    figure = go.Figure(data=data, layout=layout)
    ply.offline.iplot(figure)

  # Delta 
    delta = df[A]-df[B]
    trace0 = go.Scatter(x = df[A].index, y = delta, mode = 'lines')
    if cropTime: 
        layout = go.Layout(legend=dict(x=-.1, y=1.2) ,xaxis=dict(range=[min_date, max_date],title='Time'), yaxis=dict(zeroline=True,title='Delta',zerolinecolor='#990000',zerolinewidth=1))
    else:
        layout = go.Layout(legend=dict(x=-.1, y=1.2) ,xaxis=dict(title='Time'), yaxis=dict(zeroline=True,title='Delta',zerolinecolor='#990000',zerolinewidth=1))
    data = [trace0]
    fig = go.Figure(data=data, layout=layout)
    ply.offline.iplot(fig)
   
    # Ratio
    ratio = df[A]*1./df[B]
    trace0 = go.Scatter(x = df[A].index, y = ratio, mode = 'lines')
    if cropTime: 
        layout = go.Layout(legend=dict(x=-.1, y=1.2) ,xaxis=dict(range=[min_date, max_date],title='Time'), yaxis=dict(zeroline=True,title='Ratio',zerolinecolor='#990000',zerolinewidth=1))
    else:
        layout = go.Layout(legend=dict(x=-.1, y=1.2) ,xaxis=dict(title='Time'), yaxis=dict(zeroline=True,title='Ratio',zerolinecolor='#990000',zerolinewidth=1))
    data = [trace0]
    fig = go.Figure(data=data, layout=layout)
    ply.offline.iplot(fig)
    
    # Rolling correlation
  # fig = plt.figure(figsize=(15,6))
  # roll = mergedData.iloc[:,0].rolling(12).corr(mergedData.iloc[:,1])
  # trace0 = go.Scatter(x = df[A].index, y = roll, mode = 'lines')
  # if cropTime: 
  #     layout = go.Layout(legend=dict(x=-.1, y=1.2) ,xaxis=dict(range=[min_date, max_date],title='Time'), yaxis=dict(zeroline=True,title='Rolling Average',zerolinecolor='#990000',zerolinewidth=1))
  # else:
  #     layout = go.Layout(legend=dict(x=-.1, y=1.2) ,xaxis=dict(title='Time'), yaxis=dict(zeroline=True,title='Rolling Correlation',zerolinecolor='#990000',zerolinewidth=1))
  # data = [trace0]
  # fig = go.Figure(data=data, layout=layout)
  # ply.offline.iplot(fig)
    
    # Interactive Correlation

  # t = df[A].index
  # x = df[A]
  # y = df[B]

  # trace1 = go.Scatter(
  #     x=x, y=y, mode='markers', name='points',
  #     marker=dict(color='rgb(102,0,0)', size=2, opacity=0.4)
  # )
  # trace2 = go.Histogram2dcontour(
  #     x=x, y=y, name=df[A].column, ncontours=20,
  #     colorscale='Hot', reversescale=True, showscale=False
  # )
  # trace3 = go.Histogram(
  #     x=x, name=df[A].column,
  #     marker=dict(color='rgb(102,0,0)'),
  #     yaxis='y2'
  # )
  # trace4 = go.Histogram(
  #     y=y, name=df[B].column, marker=dict(color='rgb(102,0,0)'),
  #     xaxis='x2'
  # )
  # data = [trace1, trace2, trace3, trace4]

  # layout = go.Layout(
  #     showlegend=True,
  #     autosize=False,
  #     width=600,
  #     height=550,
  #     xaxis=dict(
  #         domain=[0, 0.85],
  #         showgrid=True,
  #         zeroline=False
  #     ),
  #     yaxis=dict(
  #         domain=[0, 0.85],
  #         showgrid=True,
  #         zeroline=False
  #     ),
  #     margin=dict(
  #         t=50
  #     ),
  #     hovermode='closest',
  #     bargap=0,
  #     xaxis2=dict(
  #         domain=[0.85, 1],
  #     showgrid=True,
  #     zeroline=False
  #     ),
  #     yaxis2=dict(
  #         domain=[0.85, 1],
  #         showgrid=True,
  #         zeroline=False
  #     )
  # )

  # fig = go.Figure(data=data, layout=layout)
  # ply.offline.iplot(fig)
    
if len(readings) < 1: print "Please load some data first..."
else:
    
    layout=widgets.Layout(width='350px')
    b_redraw = widgets.Button(description='Redraw')
    b_redraw.on_click(redraw)
    doubleAxisCheck = widgets.Checkbox(value=False, description='Secondary y axis', disabled=False)
    
    cropTimeCheck = widgets.Checkbox(value=False,description='Crop Data in X axis', disabled=False)
    minCropDate = widgets.Text(description='Start date:', layout=layout)
    maxCropDate = widgets.Text(description='End date:', layout=layout)
    
    A_kit = widgets.Dropdown(options=[k for k in readings.keys()], layout=widgets.Layout(width='350px') ,value=readings.keys()[0])
    A_kit_drop = widgets.interactive(show_sensors_A, Source=A_kit, layout=layout)
    A_sensors_drop = widgets.Dropdown(layout=widgets.Layout(width='350px'))
    show_sensors_A(readings.keys()[0])
    
    B_kit = widgets.Dropdown(options=[k for k in readings.keys()], layout=widgets.Layout(width='350px'), value=readings.keys()[1])
    B_kit_drop = widgets.interactive(show_sensors_B, Source= B_kit, layout=layout)
    B_sensors_drop = widgets.Dropdown(layout=widgets.Layout(width='350px'))
    show_sensors_B(readings.keys()[1])
    
    draw_box = widgets.HBox([b_redraw, doubleAxisCheck], layout=widgets.Layout(justify_content='space-between'))
    kit_box = widgets.HBox([A_kit, widgets.HTML('<h4><< Data source >></h4>') , B_kit], layout=widgets.Layout(justify_content='space-between'))
    sensor_box = widgets.HBox([A_sensors_drop, widgets.HTML('<h4><< Sensor selection >></h4>') , B_sensors_drop], layout=widgets.Layout(justify_content='space-between'))
    crop_box = widgets.HBox([cropTimeCheck, minCropDate, maxCropDate], layout=widgets.Layout(justify_content='space-between'))
    root_box = widgets.VBox([draw_box, kit_box, sensor_box, crop_box])
    
    display(root_box)
    
    #redraw(b_redraw)

### TODO: Anomaly Detection

Check this here https://annals-csis.org/proceedings/2012/pliks/118.pdf.

Below we'll use the Holt-Winters function as defined:

$$\hat y_{max_x}=\ell_{x−1}+b_{x−1}+s_{x−T}+m⋅d_{t−T}$$

$$\hat y_{min_x}=\ell_{x−1}+b_{x−1}+s_{x−T}-m⋅d_{t−T}$$

$$d_t=\gamma∣y_t−\hat y_t∣+(1−\gamma)d_{t−T},$$


In [ ]:
class HoltWinters:
    
    """
    Holt-Winters model with the anomalies detection using Brutlag method
    
    # series - initial time series
    # slen - length of a season
    # alpha, beta, gamma - Holt-Winters model coefficients
    # n_preds - predictions horizon
    # scaling_factor - sets the width of the confidence interval by Brutlag (usually takes values from 2 to 3)
    
    """
    
    
    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor
        
        
    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
        return sum / self.slen  
    
    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # let's calculate season averages
        for j in range(n_seasons):
            season_averages.append(sum(self.series[self.slen*j:self.slen*j+self.slen])/float(self.slen))
        # let's calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals   

          
    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []
        
        seasonals = self.initial_seasonal_components()
        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])
                
                self.PredictedDeviation.append(0)
                
                self.UpperBond.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                self.LowerBond.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                continue
                
            if i >= len(self.series): # predicting
                m = i - len(self.series) + 1
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])
                
                # when predicting we increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
                
            else:
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])
                
                # Deviation is calculated according to Brutlag algorithm.
                self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                               + (1-self.gamma)*self.PredictedDeviation[-1])
                     
            self.UpperBond.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.LowerBond.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i%self.slen])

## Data Model

### Basic Functionality
Below, we will define some basic functionality for smoothing functions.

In [ ]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import numpy as np                               # vectors and matrices
import pandas as pd                              # tables and data manipulations
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def moving_average(series, n):
    """
        Calculate average of last n observations
    """
    return np.average(series[-n:])

def exponential_smoothing(series, alpha):
    """
        series - dataset with timestamps
        alpha - float [0.0, 1.0], smoothing parameter
    """
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

def plotExponentialSmoothing(series, alphas):
    """
        Plots exponential smoothing with different alphas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters
        
    """
    with plt.style.context('seaborn-white'):    
        plt.figure(figsize=(20, 8))
        plt.plot(series.values, "c", label = "Actual")
        for alpha in alphas:
            plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
        
        plt.legend(loc="best")
        plt.axis('tight')
        plt.title("Exponential Smoothing")
        plt.grid(True);

def double_exponential_smoothing(series, alpha, beta):
    """
        series - dataset with timeseries
        alpha - float [0.0, 1.0], smoothing parameter for level
        beta - float [0.0, 1.0], smoothing parameter for trend
    """
    # first value is same as series
    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

def plotDoubleExponentialSmoothing(series, alphas, betas):
    """
        Plots double exponential smoothing with different alphas and betas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters for level
        betas - list of floats, smoothing parameters for trend
    """
    
    with plt.style.context('seaborn-white'):    
        plt.figure(figsize=(20, 8))
        plt.plot(series.values, label = "Actual")
        for alpha in alphas:
            for beta in betas:
                plt.plot(double_exponential_smoothing(series, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
        
        plt.legend(loc="best")
        plt.axis('tight')
        plt.title("Double Exponential Smoothing")
        plt.grid(True)

**Quick** demonstration of smoothing functions

In [ ]:
## Smoothing tests
dataframe = readings['CaARPAE_MAData_2_COR.csv'].dropna()
dataframe.columns

plotExponentialSmoothing(dataframe['NOX ug/m3'], [0.1])
plotDoubleExponentialSmoothing(dataframe['NOX ug/m3'], alphas=[0.05], betas=[0.02])

### Feature selection and data training split

The following code uses cross validation on rolling basis structure:

<img src="https://habrastorage.org/files/f5c/7cd/b39/f5c7cdb39ccd4ba68378ca232d20d864.png"/>

In [ ]:
from dateutil.relativedelta import relativedelta # working with dates with style
from scipy.optimize import minimize              # for function minimization

import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

## sklearn Time Series functions and data split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression

## Metrics
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error#, mean_squared_log_error 

#### TODO Select variables here 

Do Interface for variable selection and reference setting

In [ ]:
dataframe = readings['CaARPAE_MAData_2_COR.csv'].dropna()
df = dataframe[['NOX ug/m3', 'NO ug/m3']]
df.names = [['NOX', ' NO']]
print df.head(4)

#### TODO Preliminary Checks

##### TODO Dicker-fuller test (ADF)

Use this test to verify **data stationarity**.

- Null Hypothesis (H0): If failed to be rejected, it suggests the time series has a unit root, meaning it is non-stationary. It has some time dependent structure.
- Alternate Hypothesis (H1): The null hypothesis is rejected; it suggests the time series does not have a unit root, meaning it is stationary. It does not have time-dependent structure.

We interpret this result using the p-value from the test. A p-value below a threshold (such as 5% or 1%) suggests we reject the null hypothesis (stationary), otherwise a p-value above the threshold suggests we fail to reject the null hypothesis (non-stationary).

p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.

In [ ]:
ad_fuller_results = sm.tsa.stattools.adfuller(df['NOX ug/m3'])

adf = ad_fuller_results[0]
pvalue = ad_fuller_results[1]
usedlag = ad_fuller_results[2]
nobs = ad_fuller_results[3]
print 'ADF- Statistic: {}\npvalue: {}\nUsed Lag: {}\nnobs: {}\n'.format(adf, pvalue, usedlag, nobs)

##### TODO Granger Casuality Test

Use this test to determine the casuality of variables (which causes the other)

In [ ]:
print(sm.tsa.stattools.grangercausalitytests(df[['NOX ug/m3','NO ug/m3']].dropna(),1))

### TODO Naive Linear Regression

Use this only for basic checking and baseline model setting

#### Ordinary Linear Regression

In [ ]:
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots

df['const']=1
print df.head(4)

model1=sm.OLS(endog=df['NOX ug/m3'],exog=df[['NO ug/m3','const']])
results1=model1.fit()
print(results1.summary())

#### Ordinary Linear Regression with differentiation

In [ ]:
df['const']=1

df['diff_NOX ug/m3']=df['NOX ug/m3'].diff()
df['diffNO ug/m3']=df['NO ug/m3'].diff()
model2=sm.OLS(endog=df['diff_NOX ug/m3'].dropna(),exog=df[['diffNO ug/m3','const']].dropna())
results2=model1.fit()
print(results2.summary())

### TODO ARIMA(X) model

In [ ]:
Master = readings['CaARPAE_MAData_2_COR.csv'].dropna()
Slave = readings['MA04_Formulae_2.csv'].dropna()
max_date = min( Master.index[-1], Slave.index[-1])
 
mergedData = pd.merge(Master.loc[:,], Slave.loc[:,], left_index=True, right_index=True)

print mergedData.columns

In [ ]:
#Predictors = Master[['UMID %' ,'NO ug/m3', 'VV m/s', 'O3 ug/m3']]

Predictors = mergedData[['AD_2_TN_smooth (ppb)', 'AlphaDelta Humidity-%']]
to_predict = mergedData['NOX ug/m3']

TrainingSize = int(0.8*to_predict.shape[0])
TestSize = to_predict.shape[0] - TrainingSize

mod = sm.tsa.statespace.SARIMAX(to_predict[:TrainingSize],
              exog=Predictors[:TrainingSize],
              order= (2,1,3),
              enforce_invertibility=False,trend='c')
                    
res = mod.fit(disp=0)
                    
frc =res.forecast(TestSize,exog=pd.DataFrame(Predictors)[TrainingSize:])

with plt.style.context('seaborn-white'):    
    plt.figure(figsize=(20, 8))
    plt.plot(mergedData.index[TrainingSize:], frc, 'r')
    plt.plot(mergedData.index[TrainingSize:], to_predict[TrainingSize:], 'k.')
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("SARIMAX Model Prediction")
    plt.grid(True)

# R Framework

## Initialise environment

In [ ]:
## Include R in Python Notebook and test it out below - do not modify the first line (%%R -i ...)
%load_ext rpy2.ipython

In [ ]:
## CRAN mirror for use in this session
# Secure CRAN mirrors

#1: 0-Cloud [https]                   2: Algeria [https]
#3: Australia (Canberra) [https]      4: Australia (Melbourne 1) [https]
#5: Australia (Melbourne 2) [https]   6: Australia (Perth) [https]
#7: Austria [https]                   8: Belgium (Ghent) [https]
#9: Brazil (PR) [https]              10: Brazil (RJ) [https]
#11: Brazil (SP 1) [https]            12: Brazil (SP 2) [https]
#13: Bulgaria [https]                 14: Chile 1 [https]
#15: Chile 2 [https]                  16: China (Guangzhou) [https]
#17: China (Lanzhou) [https]          18: China (Shanghai) [https]
#19: Colombia (Cali) [https]          20: Czech Republic [https]
#21: Denmark [https]                  22: East Asia [https]
#23: Ecuador (Cuenca) [https]         24: Ecuador (Quito) [https]
#25: Estonia [https]                  26: France (Lyon 1) [https]
#27: France (Lyon 2) [https]          28: France (Marseille) [https]
#29: France (Montpellier) [https]     30: France (Paris 2) [https]
#31: Germany (Erlangen) [https]       32: Germany (Göttingen) [https]
#33: Germany (Münster) [https]        34: Greece [https]
#35: Iceland [https]                  36: India [https]
#37: Indonesia (Jakarta) [https]      38: Ireland [https]
#39: Italy (Padua) [https]            40: Japan (Tokyo) [https]
#41: Japan (Yonezawa) [https]         42: Korea (Ulsan) [https]
#43: Malaysia [https]                 44: Mexico (Mexico City) [https]
#45: Norway [https]                   46: Philippines [https]
#47: Serbia [https]                   48: Spain (A Coruña) [https]
#49: Spain (Madrid) [https]           50: Sweden [https]
#51: Switzerland [https]              52: Turkey (Denizli) [https]
#53: Turkey (Mersin) [https]          54: UK (Bristol) [https]
#55: UK (Cambridge) [https]           56: UK (London 1) [https]
#57: USA (CA 1) [https]               58: USA (IA) [https]
#59: USA (KS) [https]                 60: USA (MI 1) [https]
#61: USA (NY) [https]                 62: USA (OR) [https]
#63: USA (TN) [https]                 64: USA (TX 1) [https]
#65: Vietnam [https]                  66: (other mirrors)

## Install dependencies

In [ ]:
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

base = rpackages.importr('base')
utils = rpackages.importr('utils')
# select a mirror for R packages
utils.chooseCRANmirror(ind=49) # select the first mirror in the list

# R package names
packnames = ["ggplot2",
             "car",
             "lattice",
             "dyn",
             "dynlm",
             "zoo",
             "tseries",
             "lmtest",
             "xts",
             "tidyverse",
             "lubridate",
             "lme4",
             "multcomp",
             "signal",
             "ggfortify"]

# Selectively install what needs to be install.
for x in packnames:
    if not rpackages.isinstalled(x):
        utils.install_packages(StrVector(x))

# import R's "GlobalEnv" to evaluate the function
from rpy2.robjects import globalenv

# ggplot2 = rpackages.importr('ggplot2')
# graphics = rpackages.importr('graphics')

## Load in R libraries

In [ ]:
%%R

# Load in the libraries
library("ggplot2")
library("car")
library("lattice")
library("dyn")
library("dynlm")
library("zoo")
library("tseries")
library("lmtest")
library("xts")
library("tidyverse")
library("lubridate")
library("lme4")
library("multcomp")
library("signal")
library('GGally')
library('plotly')


## Export Data to R Dataframe

In [ ]:
setLimits = False
maxY = 15000
minY = 0
toshow = []
min_date_training = 0
max_date_training = 0
min_date_eval = 0
max_date_eval = 0

def show_sensors(Source):
    _sensor_drop.options = [s for s in list(readings[Source].columns)]
    _sensor_drop.source = Source
    _min_date_training.value = readings[Source].index.min()._short_repr
    _max_date_training.value = readings[Source].index.max()._short_repr
    _min_date_eval.value = readings[Source].index.min()._short_repr
    _max_date_eval.value = readings[Source].index.max()._short_repr
    
    #for k in readings.keys():
    #    _min_date_training.value = min(_min_date_training.value, readings[k].index.min()._short_repr)
    #    _max_date_training.value = max(_max_date_training.value, readings[k].index.max()._short_repr)
    #    _min_date_eval.value = min(_min_date_eval.value,readings[k].index.min()._short_repr)
    #    _max_date_eval.value = max(_max_date_eval.value,readings[k].index.max()._short_repr)
      
    
def clear_all(b):
    clear_output()
    del toshow[:]

def add_sensor(b):
    clear_output()
    d = [_sensor_drop.source, _sensor_drop.value]
    
    if d not in toshow: 
        toshow.append(d)
        
    global dataframe_export
    dataframe_export = readings[toshow[0][0]].loc[:,(toshow[0][1],)]
    
    #if b.slice_time:
    #    dataframe_export = dataframe_export[dataframe_export.index > min(_min_date_training.value,_min_date_eval.value)]
    #    dataframe_export = dataframe_export[dataframe_export.index < max(_max_date_training.value,_max_date_eval.value)]
    
    #for k in readings.keys():
    #    _min_date_training.value = min(_min_date_training.value, readings[k].index.min()._short_repr)
    #    _max_date_training.value = max(_max_date_training.value, readings[k].index.max()._short_repr)
    #    _min_date_eval.value = min(_min_date_eval.value,readings[k].index.min()._short_repr)
    #    _max_date_eval.value = max(_max_date_eval.value,readings[k].index.max()._short_repr)
    
    dataframe_export = dataframe_export[dataframe_export.index > min(_min_date_training.value,_min_date_eval.value)]
    dataframe_export = dataframe_export[dataframe_export.index < max(_max_date_training.value,_max_date_eval.value)]
    
    #print 'Min Date Training / Eval'
    #print _min_date_training.value
    #print _min_date_eval.value
    #print min(_min_date_training.value,_min_date_eval.value)
    #print 'Max Date Training / Eval'
    #print _max_date_training.value
    #print _max_date_eval.value
    #print max(_max_date_training.value,_max_date_eval.value)
    
    if len(toshow) > 1:
        for i in range(1, len(toshow)):
            dataframe_export = pd.merge(dataframe_export, readings[toshow[i][0]].loc[:,(toshow[i][1],)], left_index=True, right_index=True)

    # Change columns naming
    changed = []
    
    for i in range(len(dataframe_export.columns)):
        changed.append(toshow[i][0] + '-'+ dataframe_export.columns[i])
    dataframe_export.columns = changed
    
    #text=[i  for i in range(len(dataframe_export.columns))]
    #for i in range(len(dataframe_export.columns)):
    #    item = dataframe_export.columns[i]
    #    #print "data" + str(i)
    #    #print item
    #    fileName = item[:item.find('.')]
    #    #print fileName
    #    channel = item[item.find('-')+1:].split('-')[0]
    #    if (len(item[item.find('-')+1:].split('-'))>0):
    #        unit = item[item.find('-')+1:].split('-')[1]
    #    else:
    #        unit = ''
    #    #print channel
    #    #print unit
    #    text[i]='<br>File: '+'{:s}'+str(fileName)+'<br>Channel: '+'{:s}'+str(channel)+\
    #    '<br>Unit: '+'{:s}'+ str(unit)
    #    #print text[i]
    
    fig2 = tls.make_subplots(rows=1, cols=1, shared_xaxes=True)
    for i in range(len(dataframe_export.columns)):
        fig2.append_trace({'x': dataframe_export.index, 
                          'y': dataframe_export.iloc[:,i], 
                          'type': 'scatter',
                          'name': dataframe_export.columns[i]}, 1, 1)


    fig2['layout'].update(
        height=800,
        showlegend = True,
        legend=dict(x=-.1, y=5) ,
        xaxis=dict(
            rangeslider=dict(),
            type='date'
        ),
        annotations=[dict(
                        x=_min_date_training.value,
                        y=1,
                        xref='x',
                        yref='paper',
                        text='Training Dataset',
                        showarrow=False,
                        xanchor="left",
                        font=dict(color= 'rgba(44, 160, 101, 1)')
                    ),
                     dict(
                        x=_min_date_eval.value,
                        y=0.95,
                        xref='x',
                        yref='paper',
                        text='Evaluation Dataset',
                        showarrow=False,
                        xanchor="left",
                        font=dict(color= 'rgba(160, 160, 0, 1)')
                    )
                    ],
        shapes=[
                dict(type='rect',
                    layer='below',
                    x0=_min_date_training.value,
                    x1=_max_date_training.value,
                    y0=0.95,
                    y1=1,
                    yref= "paper",
                    fillcolor='rgba(44, 160, 0, 0.2)',
                    line=dict(color= 'rgba(44, 160, 101,0.6)'),
                    ),
                dict(type='rect',
                    layer='below',
                    x0=_min_date_eval.value,
                    x1=_max_date_eval.value,
                    y0=0.9,
                    y1=0.95,
                    yref= "paper",
                    fillcolor='rgba(160, 160, 0, 0.2)',
                    line=dict(color= 'rgba(160, 160, 0, 0.6)')
                )
        ]
    )
    
    #fig1 = dataframe_export.iplot(kind='scatter', asFigure=True, layout = layout, hoverinfo='text')
    #ply.offline.iplot(fig1)
    
    print list(dataframe_export.columns.values.tolist())
    
    ply.offline.iplot(fig2)

def reset_time_t(b):
    _min_date_training.value = readings[b.src.value].index.min()._short_repr
    _max_date_training.value = readings[b.src.value].index.max()._short_repr
    add_sensor(b)
    
def reset_time_e(b):
    _min_date_eval.value = readings[b.src.value].index.min()._short_repr
    _max_date_eval.value = readings[b.src.value].index.max()._short_repr
    add_sensor(b)

def export_dataFrame(b):
    clear_output()
    from rpy2.robjects import pandas2ri
    from rpy2.robjects import r
    
    global r_train_dataframe, train_dataframe, r_eval_dataframe, eval_dataframe

    train_dataframe = dataframe_export[dataframe_export.index > _min_date_training.value]
    train_dataframe = train_dataframe[train_dataframe.index < _max_date_training.value]
    train_dataframe.index=train_dataframe.index.to_datetime()
    r_train_dataframe = pandas2ri.py2ri(train_dataframe)
    
    eval_dataframe = dataframe_export[dataframe_export.index > _min_date_eval.value]
    eval_dataframe = eval_dataframe[eval_dataframe.index < _max_date_eval.value]
    eval_dataframe.index=eval_dataframe.index.to_datetime()
    r_eval_dataframe = pandas2ri.py2ri(eval_dataframe)
    
    %Rpush r_train_dataframe r_eval_dataframe
    
    print 'Export to R Training dataframe successful with following channels'
    %R print(colnames(r_train_dataframe))
    min_date_training = _min_date_training.value
    max_date_training= _max_date_training.value
    
    print 'With Date Range'
    print min_date_training
    print max_date_training
    
    print ''
   
    print 'Export to R Evaluation dataframe successful with following channels'
    %R print(colnames(r_eval_dataframe))
    min_date_eval = str(_min_date_eval.value)
    max_date_eval= str(_max_date_eval.value)

    print 'With Date Range'
    print min_date_eval
    print max_date_eval
        
    refFile = str(_refList.value)
    refFile = refFile[:refFile.find('.')]
    
    print ''
    
    print 'Reference Dataset'
    print refFile
    r_train_columns_renamed = False
    r_eval_columns_renamed = False
    %Rpush refFile r_train_columns_renamed r_eval_columns_renamed

_layout=widgets.Layout(width='330px')

_kit = widgets.Dropdown(options=[k for k in readings.keys()], layout=_layout)
_kit_drop = widgets.interactive(show_sensors, Source=_kit, layout=_layout)

_b_add = widgets.Button(description='Update plot', layout=widgets.Layout(width='100px'))
_b_add.on_click(add_sensor)
_b_add.slice_time = False

_sensor_drop = widgets.Dropdown(layout=_layout)
_b_reset_all = widgets.Button(description='Clear all', layout=widgets.Layout(width='100px'))
_b_reset_all.on_click(clear_all)

_b_reset_time_t = widgets.Button(description='Reset Training Dates', layout=widgets.Layout(width='200px'))
_b_reset_time_t.on_click(reset_time_t)
_b_reset_time_t.src = _kit

_b_reset_time_e = widgets.Button(description='Reset Eval Dates', layout=widgets.Layout(width='200px'))
_b_reset_time_e.on_click(reset_time_e)
_b_reset_time_e.src = _kit

_min_date_training = widgets.Text(description='Start Date Train:', layout=widgets.Layout(width='250px'))
_max_date_training = widgets.Text(description='End Date Train:', layout=widgets.Layout(width='250px'))
_min_date_eval = widgets.Text(description='Start Date Eval:', layout=widgets.Layout(width='250px'))
_max_date_eval = widgets.Text(description='End Date Eval:', layout=widgets.Layout(width='250px'))

#_b_apply_time = _b_reset = widgets.Button(description='Apply dates', layout=widgets.Layout(width='100px'))
#_b_apply_time.on_click(add_sensor)
#_b_apply_time.slice_time = True

#_b_export = widgets.Button(description='Export to CSV', layout=widgets.Layout(width='150px'))
#_b_export.on_click(export_dataFrame)
_c_R = widgets.Button(description='Export to R dataframe', layout=widgets.Layout(width='150px'))
_c_R.on_click(export_dataFrame)
#_exportPath = widgets.Text(description = 'Type in export path  ', layout=widgets.Layout(width='600px'))
#_fileName = widgets.Text(description = 'Name ', layout=widgets.Layout(width='200px'))

_button_box = widgets.HBox([_b_add, _b_reset_all])
_sensor_box = widgets.HBox([_kit_drop, _sensor_drop , _button_box])
_timeT_box = widgets.HBox([_min_date_training,_max_date_training, _b_reset_time_t])
_timeE_box = widgets.HBox([_min_date_eval,_max_date_eval, _b_reset_time_e])

#_name_box = widgets.HBox([_b_export, _exportPath, _fileName])
#_root_box = widgets.VBox([_time_box, _sensor_box, _name_box, _button_box])

_refList = widgets.RadioButtons(
    options=[k for k in readings.keys()],
    #rows=10,
    description='Reference Sensor File',
    disabled=False,
    layout=widgets.Layout(width='400px'),
)

_cR = widgets.Button(description='Export datasets to R', layout=widgets.Layout(width='200px'))
_cR.on_click(export_dataFrame)
#_prev_dataset = widgets.Button(description='Preview Datasets', layout=widgets.Layout(width='250px'))
#_prev_dataset.on_click(preview_datasets)
_button_box = widgets.HBox([_refList,_cR])
_root_box = widgets.VBox([_sensor_box, _timeT_box, _timeE_box, _button_box])
display(widgets.HTML('<br>'))

display(widgets.HTML('<h3>Use this box to create R compatible dataframes</h3>'))
display(widgets.HTML('1. Select the signals from each source, and hit Preview Slice'))
display(widgets.HTML('2. Apply dates from and to export trim'))
display(widgets.HTML('3. Hit export to DataFrame'))
display(widgets.HTML('<br>'))
display(_root_box)



### Renaming and timestamp reading

In [ ]:
%%R

#print(r_train_columns_renamed)
#print(refFile)

convertNames <- function(dataset){
    for (i in colnames(dataset)){
        #print(i)
        index <- gregexpr('.csv.',i, fixed = TRUE)
        #print(index)
        #print('-')
        fileName=substr(i, start=1, stop=index)
        fileName=substr(fileName,start=1,stop=nchar(fileName)-1)
        channel=substr(i, start=nchar(fileName)+6, stop=nchar(i))
        #print('FileName')
        #print(fileName)
        #print('Channel')
        #print(channel)
        
        if (fileName==refFile){
            #fileName=paste('REF',fileName,sep="-")
            fileName='REF'
            #print('Reference Dataset')
        } else {
            #fileName='CORR'
        }

        if (grepl('Carbon.monoxide.kOhm.ppm.sm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Carbon.monoxide.kOhm.ppm.sm',channel),nchar(channel)),'KIT_CO_RAW_SM',channel),sep="_")
        } else if (grepl('Carbon.monoxide.kOhm.ppm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Carbon.monoxide.kOhm.ppm',channel),nchar(channel)),'KIT_CO_RAW',channel),sep="_")
        } else if (grepl('Carbon.monoxide.sm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Carbon.monoxide.sm',channel),nchar(channel)),'KIT_CO_PPM_SM',channel),sep="_")
        } else if (grepl('Carbon.monoxide',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Carbon.monoxide',channel),nchar(channel)),'KIT_CO_PPM',channel),sep="_")
        } else if (grepl('Nitrogen.dioxide.kOhm.ppm.sm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Nitrogen.dioxide.kOhm.ppm.sm',channel),nchar(channel)),'KIT_NO2_RAW_SM',channel),sep="_")
        } else if (grepl('Nitrogen.dioxide.kOhm.ppm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Nitrogen.dioxide.kOhm.ppm',channel),nchar(channel)),'KIT_NO2_RAW',channel),sep="_")
        } else if (grepl('Nitrogen.dioxide.sm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Nitrogen.dioxide.sm',channel),nchar(channel)),'KIT_NO2_PPM_SM',channel),sep="_")
        } else if (grepl('Nitrogen.dioxide',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Nitrogen.dioxide',channel),nchar(channel)),'KIT_NO2_PPM',channel),sep="_")
        } else if (grepl('AlphaDelta.Humidity..',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.Humidity..',channel),nchar(channel)),'AD_H_PRCT',channel),sep="_")
        } else if (grepl('AlphaDelta.Humidity',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.Humidity',channel),nchar(channel)),'AD_H_PRCT',channel),sep="_")
        } else if (grepl('Humidity..',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Humidity..',channel),nchar(channel)),'KIT_H_PRCT',channel),sep="_")
        } else if (grepl('Humidity',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Humidity',channel),nchar(channel)),'KIT_H_PRCT',channel),sep="_")
        } else if (grepl('AlphaDelta.Humidity',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.Humidity',channel),nchar(channel)),'KIT_H_PRCT',channel),sep="_")
        } else if (grepl('AlphaDelta.Temperature.C',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.Temperature.C',channel),nchar(channel)),'AD_T_C',channel),sep="_")
        } else if (grepl('AlphaDelta.Temperature',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.Temperature',channel),nchar(channel)),'AD_T_C',channel),sep="_")
        } else if (grepl('Temperature.C',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Temperature.C',channel),nchar(channel)),'KIT_T_C',channel),sep="_")
        } else if (grepl('Temperature',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Temperature.C',channel),nchar(channel)),'KIT_T_C',channel),sep="_")
        } else if (grepl('Battery..',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('Battery..',channel),nchar(channel)),'BATT_R',channel),sep="_")
        } else if (grepl('AlphaDelta.1W.mV',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.1W.mV',channel),nchar(channel)),'AD_1W_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.1A.mV',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.1A.mV',channel),nchar(channel)),'AD_1A_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.2W.mV',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.2W.mV',channel),nchar(channel)),'AD_2W_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.2A.mV',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.2A.mV',channel),nchar(channel)),'AD_2A_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.3W.mV',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.3W.mV',channel),nchar(channel)),'AD_3W_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.3A.mV',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.3A.mV',channel),nchar(channel)),'AD_3A_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.3A.mV',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.3A.mV',channel),nchar(channel)),'AD_3A_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.1cal.ppm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.1cal.ppm',channel),nchar(channel)),'AD_CO_PPM_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.2cal.ppm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.2cal.ppm',channel),nchar(channel)),'AD_NO2_PPM_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.3cal.ppm',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.3cal.ppm',channel),nchar(channel)),'AD_O3_PPM_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta_CO_SM',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta_CO_SM',channel),nchar(channel)),'AD_CO_PPM_SM',channel),sep="_")
        } else if (grepl('AlphaDelta.CO',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.CO',channel),nchar(channel)),'AD_CO_PPM_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta_NO2_SM',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta_NO2_smooth',channel),nchar(channel)),'AD_NO2_PPM_SM',channel),sep="_")
        } else if (grepl('AlphaDelta.NO2',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.NO2',channel),nchar(channel)),'AD_NO2_PPM_RAW',channel),sep="_")
        } else if (grepl('AlphaDelta.OX',channel)){
            newName = paste(fileName,gsub(channel,substr(channel,regexpr('AlphaDelta.OX',channel),nchar(channel)),'AD_O3_PPM_RAW',channel),sep="_")
        }

        #print(newName)
        #name[i]<-newName
        #colnames(r_train_dataframe) <- paste(fileName,"-",channel)
        colnames(dataset)[colnames(dataset) == i] <- newName
        #print('----')
    }
    return(dataset)
}

if (!r_train_columns_renamed){
    print('Time format for Training Dataset')
    r_train_dataframe = read.zoo(r_train_dataframe, index = "Time",
      format = "%Y-%m-%d %H:%M:00", tz = "GMT+2")
    print(time(r_train_dataframe)[1])

    r_train_dataframe=convertNames(r_train_dataframe)
    r_train_columns_renamed = TRUE 
}

if (!r_eval_columns_renamed){
    print('Time format for Evaluation Dataset')
    r_eval_dataframe = read.zoo(r_eval_dataframe, index = "Time",
      format = "%Y-%m-%d %H:%M:00", tz = "GMT+2")
    print(time(r_eval_dataframe)[1])
    r_eval_dataframe=convertNames(r_eval_dataframe)
    r_eval_columns_renamed = TRUE 
}

print('Renamed Training Dataset Columns')
print(colnames(r_train_dataframe))
print('Renamed Eval Dataset Columns')
print(colnames(r_eval_dataframe))


## Data Exploration

### Pairs Plot

In [ ]:
%%R -w 800 -h 500
print(colnames(r_train_dataframe))

In [ ]:
%%R -w 800 -h 600
options(warn=-1)
fileNamePairs = 'KIT_1_4574_MICS'

p_train<-ggpairs(data=r_train_dataframe, # data.frame with variables
             columns=c(2,3,4,6,8), # columns to plot, default to all.
             title="Pairs Plot For Training Dataset")
pp_train<-p_train+theme(axis.text=element_text(size=8),
        axis.title=element_text(size=6))
print(pp_train)

#pp_train_plotly<-ggplotly(pp_train)
#print(pp_train_plotly)

p_eval<-ggpairs(data=r_eval_dataframe, # data.frame with variables
             columns=c(2,3,4), # columns to plot, default to all.
             title="Pairs Plot For Eval Dataset")
pp_eval<-p_eval+theme(axis.text=element_text(size=8),
        axis.title=element_text(size=6))
print(pp_eval)

#pp_eval_plotly <- ggplotly(pp_eval)
#print(pp_eval_plotly)

options(warn=0)

### Eval DataFrame
#pairs(~r_eval_dataframe[,"REF_KIT_CO_RAW"]+
#      r_eval_dataframe[,"REF_KIT_NO2_RAW"]+
#      r_eval_dataframe[,paste(fileNamePairs,"KIT_CO_RAW",sep="_")])

### Coplot

In [ ]:
%%R -w 700 -h 700

p<-coplot(r_train_dataframe[,"REF_AD_CO_PPM_SM"] ~ 
          r_train_dataframe[,"KIT_1_4574_KIT_CO_RAW_SM"] | 
          r_train_dataframe[,"KIT_1_4574_KIT_H_PRCT"] + 
          r_train_dataframe[,"REF_AD_T_PRCT"])

## Model Iterations

In [ ]:
%%R
print(colnames(r_train_dataframe))

In [ ]:
%%R -w 800 -h 500

library(ggfortify)

# Model 1 Names - Specify var and model names below
r_mod1_ref = "REF_AD_NO2_PPM_SM"
r_train_dataframe.mod1.name = "NO2_MICS_O(1)"
# Model 1 Formulation - Specify model below
r_train_dataframe.mod1 = dynlm(REF_AD_NO2_PPM_SM ~ KIT_1_4574_KIT_NO2_RAW_SM, 
                               data = r_train_dataframe)

# Model Reference
r_train_dataframe.mod1.reference= r_train_dataframe[,r_mod1_ref]
r_train_dataframe.mod1.reference.name = r_mod1_ref

print(summary(r_train_dataframe.mod1))

# Use Jarque Bera Test to check if the regression errors are normally 
# distributed for DW test (assumes that the regression errors are normally distributed)
# The p-value is the probability of the null hypotesis 
# (which for the Jarque Bera Test is that the distribution is normal)
print(jarque.bera.test(resid(r_train_dataframe.mod1)))

plot(resid(r_train_dataframe.mod1), xlab = "Date", main ="", type = "o")

# Check for autocorrelation of the residuals, 
# with DW test if the residuals are normally distributed
print(dwtest((r_train_dataframe.mod1), alternative = "two.sided"))

par(mfrow = c(2, 2), oma = c(0, 0, 2, 0))
autoplot(r_train_dataframe.mod1)
# https://stats.stackexchange.com/questions/58141/interpreting-plot-lm#65864

In [ ]:
%%R -w 800 -h 500

library(ggfortify)

# Model 1 Names - Specify var and model names below
r_mod2_ref = "REF_AD_NO2_PPM_SM"
r_train_dataframe.mod2.name = "NO2_MICS_O(2)"
# Model 1 Formulation - Specify model below
r_train_dataframe.mod2 = dynlm(REF_AD_NO2_PPM_SM ~ KIT_1_4574_KIT_NO2_RAW_SM+I(KIT_1_4574_KIT_NO2_RAW_SM^2), 
                               data = r_train_dataframe)

# Model Reference
r_train_dataframe.mod2.reference= r_train_dataframe[,r_mod2_ref]
r_train_dataframe.mod2.reference.name = r_mod2_ref

print(summary(r_train_dataframe.mod2))

# Use Jarque Bera Test to check if the regression errors are normally 
# distributed for DW test (assumes that the regression errors are normally distributed)
# The p-value is the probability of the null hypotesis 
# (which for the Jarque Bera Test is that the distribution is normal)
print(jarque.bera.test(resid(r_train_dataframe.mod2)))

plot(resid(r_train_dataframe.mod2), xlab = "Date", main ="", type = "o")

# Check for autocorrelation of the residuals, 
# with DW test if the residuals are normally distributed
print(dwtest((r_train_dataframe.mod2), alternative = "two.sided"))

par(mfrow = c(2, 2), oma = c(0, 0, 2, 0))
autoplot(r_train_dataframe.mod2)
# https://stats.stackexchange.com/questions/58141/interpreting-plot-lm#65864

In [ ]:
%%R -w 800 -h 500

library(ggfortify)

# Model 1 Names - Specify var and model names below
r_mod3_ref = "REF_AD_NO2_PPM_SM"
r_train_dataframe.mod3.name = "NO2_MICS_O(1) + H"
# Model 1 Formulation - Specify model below
r_train_dataframe.mod3 = dynlm(REF_AD_NO2_PPM_SM ~ KIT_1_4574_KIT_NO2_RAW_SM + KIT_1_4574_KIT_H_PRCT, 
                               data = r_train_dataframe)

# Model Reference
r_train_dataframe.mod3.reference= r_train_dataframe[,r_mod3_ref]
r_train_dataframe.mod3.reference.name = r_mod3_ref

print(summary(r_train_dataframe.mod3))

# Use Jarque Bera Test to check if the regression errors are normally 
# distributed for DW test (assumes that the regression errors are normally distributed)
# The p-value is the probability of the null hypotesis 
# (which for the Jarque Bera Test is that the distribution is normal)
print(jarque.bera.test(resid(r_train_dataframe.mod3)))

plot(resid(r_train_dataframe.mod3), xlab = "Date", main ="", type = "o")

# Check for autocorrelation of the residuals, 
# with DW test if the residuals are normally distributed
print(dwtest((r_train_dataframe.mod3), alternative = "two.sided"))

par(mfrow = c(2, 2), oma = c(0, 0, 2, 0))
autoplot(r_train_dataframe.mod3)
# https://stats.stackexchange.com/questions/58141/interpreting-plot-lm#65864

In [ ]:
%%R -w 800 -h 500

library(ggfortify)

# Model 1 Names - Specify var and model names below
r_mod4_ref = "REF_AD_NO2_PPM_SM"
r_train_dataframe.mod4.name = "NO2_MICS_O(1) + T"
# Model 1 Formulation - Specify model below
r_train_dataframe.mod4 = dynlm(REF_AD_NO2_PPM_SM ~ KIT_1_4574_KIT_NO2_RAW_SM + KIT_1_4574_KIT_T_C, 
                               data = r_train_dataframe)

# Model Reference
r_train_dataframe.mod4.reference= r_train_dataframe[,r_mod4_ref]
r_train_dataframe.mod4.reference.name = r_mod4_ref

print(summary(r_train_dataframe.mod4))

# Use Jarque Bera Test to check if the regression errors are normally 
# distributed for DW test (assumes that the regression errors are normally distributed)
# The p-value is the probability of the null hypotesis 
# (which for the Jarque Bera Test is that the distribution is normal)
print(jarque.bera.test(resid(r_train_dataframe.mod4)))

plot(resid(r_train_dataframe.mod4), xlab = "Date", main ="", type = "o")

# Check for autocorrelation of the residuals, 
# with DW test if the residuals are normally distributed
print(dwtest((r_train_dataframe.mod4), alternative = "two.sided"))

par(mfrow = c(2, 2), oma = c(0, 0, 2, 0))
autoplot(r_train_dataframe.mod4)
# https://stats.stackexchange.com/questions/58141/interpreting-plot-lm#65864

In [ ]:
%%R -w 800 -h 500

library(ggfortify)

# Model 1 Names - Specify var and model names below
r_mod5_ref = "REF_AD_NO2_PPM_SM"
r_train_dataframe.mod5.name = "NO2_MICS_O(1) + T + H"
# Model 1 Formulation - Specify model below
r_train_dataframe.mod5 = dynlm(REF_AD_NO2_PPM_SM ~ KIT_1_4574_KIT_NO2_RAW_SM + KIT_1_4574_KIT_T_C+KIT_1_4574_KIT_H_PRCT, 
                               data = r_train_dataframe)

# Model Reference
r_train_dataframe.mod5.reference= r_train_dataframe[,r_mod5_ref]
r_train_dataframe.mod5.reference.name = r_mod5_ref

print(summary(r_train_dataframe.mod5))

# Use Jarque Bera Test to check if the regression errors are normally 
# distributed for DW test (assumes that the regression errors are normally distributed)
# The p-value is the probability of the null hypotesis 
# (which for the Jarque Bera Test is that the distribution is normal)
print(jarque.bera.test(resid(r_train_dataframe.mod5)))

plot(resid(r_train_dataframe.mod5), xlab = "Date", main ="", type = "o")

# Check for autocorrelation of the residuals, 
# with DW test if the residuals are normally distributed
print(dwtest((r_train_dataframe.mod5), alternative = "two.sided"))

par(mfrow = c(2, 2), oma = c(0, 0, 2, 0))
autoplot(r_train_dataframe.mod5)
# https://stats.stackexchange.com/questions/58141/interpreting-plot-lm#65864

### Model Diagnostics

More into detail for model diagnostics [here](https://www.statmethods.net/stats/rdiagnostics.html)
and [here](https://socialsciences.mcmaster.ca/jfox/Courses/Brazil-2009/index.html)

All the plots explanations are [here]( https://stats.stackexchange.com/questions/5135/interpretation-of-rs-lm-output).
As a rule of a thumb: if Pr(>|t|) is very small, means the value is significant - P-value < 0.05 is OK.

Use **Jarque Bera Test** to check if the regression errors are normally 
distributed for DW test (assumes that the regression errors are normally distributed). The p-value is the probability of the null hypotesis (which for the Jarque Bera Test is that the distribution is normal)

Check for autocorrelation of the residuals, with **DW Test** if the residuals are normally distributed: https://stats.stackexchange.com/questions/14914/how-to-test-the-autocorrelation-of-the-residuals

Here more information about the plots. [Link](https://stats.stackexchange.com/questions/58141/interpreting-plot-lm#65864)

General rules:

- **Residual vs Fitted**: we want a horizontal red line with homogeneus spread. It's a check for the heterodasticity of the distribution. If the residuals change is correlated with the fitted values or the original, our model assumptions are NOK (see https://stats.stackexchange.com/questions/76226/interpreting-the-residuals-vs-fitted-values-plot-for-verifying-the-assumptions)
- **Scale location**: we want the red line horizontal - same check as above (see https://stats.stackexchange.com/questions/52089/what-does-having-constant-variance-in-a-linear-regression-model-mean/52107#52107)
- **Normal QQ**: You can interpret a qq-plot analytically by considering the values read from the axes compare for a given plotted point. If the data were well described by a normal distribution, the values should be about the same. For example, take the extreme point at the very far left bottom corner: its x value is somewhere past −3, but its y value is only a little past −.2, so it is much further out than it 'should' be. In general, a simple rubric to interpret a qq-plot is that if a given tail twists off counterclockwise from the reference line, there is more data in that tail of your distribution than in a theoretical normal, and if a tail twists off clockwise there is less data in that tail of your distribution than in a theoretical normal. In other words:

    - if both tails twist counterclockwise you have heavy tails (leptokurtosis),
    - if both tails twist clockwise, you have light tails (platykurtosis),
    - if your right tail twists counterclockwise and your left tail twists clockwise, you have - right skew
    - if your left tail twists counterclockwise and your right tail twists clockwise, you have left skew

In [ ]:
%%R

## Only done for model #3
#options(repr.plot.width=6, repr.plot.height=5)
## Outlier tests
#print('Outlier Tests')
#outlierTest(r_train_dataframe.mod1)
#leveragePlots(r_train_dataframe.mod1)
#qqPlot(r_train_dataframe.mod1)
#
## Evaluate homoscedasticity
## non-constant error variance test
#print('Homoscedasticity Tests')
#ncvTest(r_train_dataframe.mod1)
## plot studentized residuals vs. fitted values 
#spreadLevelPlot(r_train_dataframe.mod1)
#
## Evaluate Nonlinearity
## component + residual plot 
#print('Nonlinearity Tests')
#crPlots(r_train_dataframe.mod1)
## Ceres plots 
#ceresPlots(r_train_dataframe.mod1)
#
## Evaluate Collinearity
#print('Nonlinearity Tests')
#vif(r_train_dataframe.mod1) # variance inflation factors 
#sqrt(vif(r_train_dataframe.mod1)) > 2 # problem?

### Model Fit Plot

Use the cells below to plot model data.

#### Using Traditional R

In [ ]:
%%R -w 800 -h 500

plot(r_train_dataframe.mod1.reference ,col="black")
grid(5, 5, lwd = 0.85) # grid only in y-direction
points(r_train_dataframe.mod1$index, r_train_dataframe.mod1$fit, col="red", type='b')
points(r_train_dataframe.mod2$index, r_train_dataframe.mod2$fit, col="green", type='b')
points(r_train_dataframe.mod3$index, r_train_dataframe.mod3$fit, col="blue", type='b')
points(r_train_dataframe.mod4$index, r_train_dataframe.mod4$fit, col="yellow", type='b')
points(r_train_dataframe.mod5$index, r_train_dataframe.mod5$fit, col="gray", type='b')


legend('topright', 
       legend=c(r_train_dataframe.mod1.reference.name, 
                r_train_dataframe.mod1.name,
                r_train_dataframe.mod2.name, 
                r_train_dataframe.mod3.name,
                r_train_dataframe.mod4.name,                
                r_train_dataframe.mod5.name), 
       col=c("black", "red", "green", "blue", "yellow", "gray"), lty=1:6, cex=0.8)


#### Using interactive Plot

In [ ]:
%%R

# Plot data fitting with respect to sample
p1 <- plot_ly()

## Reference
p1 <- add_trace(p1, 
                x = time(r_train_dataframe), 
                y = ~r_train_dataframe.mod1.reference, 
                name = r_train_dataframe.mod1.reference.name, mode = 'lines')

## Model Fit 1
p1 <- add_trace(p1, 
                x = time(r_train_dataframe.mod1), 
                y = ~r_train_dataframe.mod1$fit,
               name = r_train_dataframe.mod1.name, mode = 'lines')

### Model Fit 2
p1 <- add_trace(p1, 
                x = time(r_train_dataframe.mod2), 
                y = ~r_train_dataframe.mod2$fit,
               name = r_train_dataframe.mod2.name, mode = 'lines')

## Model Fit 3
p1 <- add_trace(p1, 
                x = time(r_train_dataframe.mod3), 
                y = ~r_train_dataframe.mod3$fit,
               name = r_train_dataframe.mod3.name, mode = 'lines')

### Model Fit 4
#p1 <- add_trace(p1, 
#                x = time(r_train_dataframe.mod4), 
#                y = ~r_train_dataframe.mod4$fit,
#               name = r_train_dataframe.mod4.name)

print(p1)

In [ ]:
%%R
install.packages('h2o')